In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#from utils_new import extract_PID_data, resample_PID_data, Throttle_Constraint, RPM_Constraint, Speed_Constraint, Remove_Outliers
import os
import json
from operator import itemgetter
import seaborn as sns
#from scipy.stats import pearson
from sklearn.metrics.pairwise import cosine_similarity
from scipy.spatial import distance
import plotly.graph_objects as go
#from datetime import  timezone
import datetime
from plotly.subplots import make_subplots
from scipy.stats import norm
from scipy import signal
import plotly.express as px
import requests
import time

In [ ]:
import requests

# Define the URL for the API endpoint
url = "https://algo-internal-apis.intangles-aws-us-east-1.intangles.us/dtc/info"

# Define the dtcs parameter as a list of DTC IDs
dtcs = ["P0404", "P0195", "P02DD"]  # Replace with actual DTC IDs

# Construct the request body
body = {
    "dtcs": dtcs
}

# Send the POST request
try:
    response = requests.post(url, json=body)

    # Check the response status
    if response.status_code == 200:
        print("Success:", response.json())
    else:
        print("Error:", response.status_code, response.json())
except Exception as e:
    print("An error occurred:", str(e))

In [ ]:
import requests

# Define the URL for the API endpoint
url = "https://algo-internal-apis.intangles-aws-us-east-1.intangles.us/dtc/info"

# Define the dtcs parameter as a list of DTC IDs
body = {
    "dtcs": {
        "codes": ["P0404", "P0195", "P02DD"]
    }
}

# Construct the request body
body = {
    "dtcs": dtcs
}

# Print the request body for debugging
print("Request Body:", body)

# Send the POST request
try:
    response = requests.post(url, json=body)

    # Check the response status
    if response.status_code == 200:
        print("Success:", response.json())
    else:
        print("Error:", response.status_code, response.json())
except Exception as e:
    print("An error occurred:", str(e))


In [ ]:
url = "https://algo-internal-apis.intangles-aws-us-east-1.intangles.us/dtc/info/"

In [ ]:
dtc_response = requests.post(url)

In [ ]:
dtc_response.json()

In [ ]:
dtcs = [" P0195 "]

In [ ]:

 
headers = {
                'Content-Type': 'application/json',
            }
vehicle_dtc_data =  {"dtcs": dtcs}
 

In [ ]:
dtc_response = requests.post(url, json=vehicle_dtc_data, headers=headers)

In [ ]:
dtc_response.json()

In [ ]:
def extract_PID_data(data, PROTOCOL,LABEL):
    
    if (PROTOCOL == 'SAE_avg'):

        if LABEL == 'IMAP':
            PID_TAG = 'spn_106_avg'
        elif LABEL == 'ENGINE LOAD':
            PID_TAG = 'spn_92_avg'
        elif LABEL == 'ENGINE RPM':
            PID_TAG = 'spn_190_avg'
        elif LABEL == 'FUEL RATE':
            PID_TAG = 'spn_183_avg'
        elif LABEL == 'MAF':
            PID_TAG = 'spn_132_avg'
        elif LABEL == 'BOOST':
            PID_TAG = 'spn_102_avg'
        elif LABEL == 'BAROMETER':
            PID_TAG = 'spn_108_avg'
        elif LABEL == 'THROTTLE':
            PID_TAG = 'spn_91_avg'
        elif LABEL == 'ATGMF': # Eaxuast Gas Flow Rate
            PID_TAG = 'spn_3236_avg'
        elif LABEL == 'DPFDP': # DPF Diffrential Pressure (across DPF)
            PID_TAG = 'spn_3251_avg'
        elif LABEL == 'SCRT':   # SCR Catalyst Temperature Before Catalyst (DPF out)
            PID_TAG = 'spn_4360_avg'
        elif LABEL == 'SPEED': # Wheep based Vehicle Speed
            PID_TAG = 'spn_84_avg'
        elif LABEL == 'DPFINT':# DPF in Temperature Before DPF (DOC out)
            PID_TAG = 'spn_3250_avg' #4766
        elif LABEL == 'IS': #  regen inhibited
            PID_TAG = 'spn_3703_avg'        
        elif LABEL == 'FUEL USE': #  Total fuel used (high precision)
            PID_TAG = 'spn_5054_avg'        
        elif LABEL == 'DISTANCE': #  Total distance travelled (high precision)
            PID_TAG = 'spn_917_avg' # 245
        elif LABEL == 'SOOTLOAD_3719':
            PID_TAG = 'spn_3719_avg' # 245 #3719 generic check  
        elif LABEL == 'SOOTLOAD_5466':
            PID_TAG = 'spn_5466_avg' # 245 #3719 generic chec         
        elif LABEL == 'ACTIVEREGEN':
            PID_TAG = 'spn_3700_avg' # 245
        elif LABEL == 'PEDAL':
            PID_TAG = 'spn_91_avg' # 245  
        elif LABEL == 'LOAD':
            PID_TAG = 'spn_92_avg' # 245      

    elif(PROTOCOL == 'ISO'):

        if LABEL == 'IMAP':
            PID_TAG = '87BC'#MODIFY the "if PID_TAG in State1:" loop (append for loop on top)  
        elif LABEL == 'ENGINE LOAD':
            PID_TAG = '04'
        elif LABEL == 'ENGINE RPM':
            PID_TAG = '0C'
        elif LABEL == 'FUEL RATE':
            PID_TAG = '5E'
        elif LABEL == 'MAF':
            PID_TAG = '10'
        elif LABEL == 'BOOST':
            PID_TAG = '102'
        elif LABEL == 'BAROMETER':
            PID_TAG = '33'
        elif LABEL == 'THROTTLE':
            PID_TAG = '49'#MODIFY the "if PID_TAG in State1:" loop (append for loop on top)
        elif LABEL == 'DPFDP': # DPF Diffrential Pressure (across DPF)
            PID_TAG = '7A'
        elif LABEL == 'DPFDP': # DPF Diffrential Pressure (across DPF)
            PID_TAG = '7A'    

    #print(LABEL)
    print("PID TAG is-->" + LABEL + " ," +PROTOCOL,"Mapping is --->", PID_TAG)

   

    Time_vec = []
    Val_vec = []
    #print(len(data[0]))
    #print(len(data))

    for data_cnt in range(0,len(data)):
        if "pids" in data[data_cnt]:
            if len(data[data_cnt]['pids'])>0:
                for sub_pid_cnt in range(0,len(data[data_cnt]['pids'])):  #this loop
                    State = data[data_cnt]['pids'][sub_pid_cnt]
                    #print(State)
                    #print(data_cnt)
                    #for state_cnt in range(0,len(State)):
                    if PID_TAG in State:
                        #print("--------------------------------------------IN----------------------------------")
                        Time_vec.append(np.array(State[PID_TAG]['timestamp'], dtype=np.int64))
                        Val_vec.append(np.array(State[PID_TAG]['value'], dtype=float))

    print("Number of time stamp available are--->",len(Val_vec))

    
    return Time_vec,Val_vec

In [ ]:
Start_TS =  1716834600000
End_TS = 1717266540000
vehicle = '286461304969625600'

# OBD_data_path = "C:/Users/Harleen.Kaur/Documents/EBT_VEHICLES_FOR_TESTING/1286430381523861504_1716316200000_1717525740000"


# with open(OBD_data_path, 'r') as file:
#         OBD_data = json.load(file)
OBD_data_path = 'https://old-data-downloader.intangles-aws-us-east-1.intangles.us/download/' + str(vehicle) +  "/" + str(Start_TS) + "/" + str(End_TS)
r = requests.get(OBD_data_path, stream=True)
OBD_data = r.json()

#C:\Users\Harleen.Kaur\Downloads\Get_Thresholds.py

# OBD_data_path = 'https://old-data-downloader.intangles-aws-us-east-1.intangles.us/download/' + str(vehicle) +  "/" + str(Start_TS) + "/" + str(End_TS)
# r = requests.get(OBD_data_path, stream=True)
# OBD_data = r.json()

In [ ]:
import plotly.graph_objects as go
import plotly.subplots as subplots

# Create the figure
fig = subplots.make_subplots(rows=7, cols=1, shared_xaxes=True)


X_Time, X_Value = extract_PID_data(OBD_data,'SAE','DPFDP')
DP_val = []
TS = []
for cnt in range(0,len(X_Time)):
    TEMP = datetime.datetime.fromtimestamp((X_Time[cnt] + 19800*1000) / 1000.0, tz=datetime.timezone.utc)
    TS.append(TEMP)
    DP_val.append(X_Value[cnt][0])

# Add the first trace
fig.add_trace(go.Scatter(x=TS, y=DP_val, name='DPFDP', mode='markers' ), row=1, col=1 )

fig.add_trace(go.Scatter(x=TS, y=[7]*len(TS), name='Horizontal Line', mode='lines', line=dict(color='red', dash='dash')), row=1, col=1 )


X_Time, X_Value = extract_PID_data(OBD_data,'SAE','DPFINT')
Temp_val = []
TS = []
for cnt in range(0,len(X_Time)):
    TEMP = datetime.datetime.fromtimestamp((X_Time[cnt] + 19800*1000) / 1000.0, tz=datetime.timezone.utc)
    TS.append(TEMP)
    Temp_val.append(X_Value[cnt][0])


# Add the second trace
fig.add_trace(go.Scatter(x=TS, y=Temp_val, name='DPFINT'), row=2, col=1)   

fig.add_trace(go.Scatter(x=TS, y=[500]*len(TS), name='Horizontal Line', mode='lines', line=dict(color='red', dash='dash')), row=2, col=1 )



X_Time, X_Value = extract_PID_data(OBD_data,'SAE','ENGINE RPM')
Speed_val = []
TS = []
for cnt in range(0,len(X_Time)):
    TEMP = datetime.datetime.fromtimestamp((X_Time[cnt] + 19800*1000) / 1000.0, tz=datetime.timezone.utc)
    TS.append(TEMP)
    Speed_val.append(X_Value[cnt][0])

# Add the second trace
fig.add_trace(go.Scatter(x=TS, y=Speed_val, name='ENGINE RPM'), row=3, col=1)


X_Time, X_Value = extract_PID_data(OBD_data,'SAE','SPEED')
Speed_val = []
TS = []
for cnt in range(0,len(X_Time)):
    TEMP = datetime.datetime.fromtimestamp((X_Time[cnt] + 19800*1000) / 1000.0, tz=datetime.timezone.utc)
    TS.append(TEMP)
    Speed_val.append(X_Value[cnt][0])

# Add the second trace
fig.add_trace(go.Scatter(x=TS, y=Speed_val, name='SPEED'), row=4, col=1)



X_Time, X_Value = extract_PID_data(OBD_data,'SAE','SOOTLOAD_5466')
AR_val = []
TS = []
for cnt in range(0,len(X_Time)):
    TEMP = datetime.datetime.fromtimestamp((X_Time[cnt] + 19800*1000) / 1000.0, tz=datetime.timezone.utc)
    TS.append(TEMP)
    AR_val.append(X_Value[cnt][0])

# Add the second trace
fig.add_trace(go.Scatter(x=TS, y=AR_val, name='SOOTLOAD_5466'), row=5, col=1)  


X_Time, X_Value = extract_PID_data(OBD_data,'SAE','SOOTLOAD_3719')
AR_val = []
TS = []
for cnt in range(0,len(X_Time)):
    TEMP = datetime.datetime.fromtimestamp((X_Time[cnt] + 19800*1000) / 1000.0, tz=datetime.timezone.utc)
    TS.append(TEMP)
    AR_val.append(X_Value[cnt][0])

# Add the second trace
fig.add_trace(go.Scatter(x=TS, y=AR_val, name='SOOTLOAD_3719'), row=6, col=1)    




X_Time, X_Value = extract_PID_data(OBD_data,'SAE','ACTIVEREGEN')
AR_val = []
TS = []
for cnt in range(0,len(X_Time)):
    TEMP = datetime.datetime.fromtimestamp((X_Time[cnt] + 19800*1000) / 1000.0, tz=datetime.timezone.utc)
    TS.append(TEMP)
    AR_val.append(X_Value[cnt][0])

# Add the second trace
fig.add_trace(go.Scatter(x=TS, y=AR_val, name='ACTIVEREGEN', mode='markers'), row=7, col=1)    


# Set the y-axis titles
fig.update_layout(yaxis1_title='DPFDP', yaxis2_title='DPFINT', yaxis3_title='ENGINE RPM', yaxis4_title='SPEED', yaxis5_title='SOOTLOAD_5466',
                  yaxis6_title='SOOTLOAD_3719',
                  yaxis7_title='ACTIVEREGEN')    


# X_Time, X_Value = extract_PID_data(OBD_data,'SAE','SPEED')
# Speed_val = []
# TS = []
# for cnt in range(0,len(X_Time)):
#     TEMP = datetime.datetime.fromtimestamp((X_Time[cnt] + 19800*1000) / 1000.0)
#     TS.append(TEMP)
#     Speed_val.append(X_Value[cnt][0])


# Show the plot
fig.show()



#fig.write_html("C:/Users/Harleen.Kaur/Documents/EBT_VEHICLES_FOR_TESTING/RWTL/" + "1286430381523861504_1716316200000_1717525740000.html")

In [ ]:
X_Time, X_Value = extract_PID_data(OBD_data,'SAE','SOOTLOAD_5466')
SL_val = []
TS = []
for cnt in range(0,len(X_Time)):
    TEMP = datetime.datetime.fromtimestamp((X_Time[cnt] + 19800*1000) / 1000.0, tz=datetime.timezone.utc)
    TS.append(TEMP)
    SL_val.append(X_Value[cnt][0])

In [ ]:
relevant_idx = (np.abs(np.asarray(X_Time) - (1716575400000))).argmin()

In [ ]:
difference = []
for i in range(relevant_idx + 1, len(SL_val)):
    difference.append(SL_val[i]-SL_val[i-1])


In [ ]:
difference = []
for i in range(1, len(SL_val)-1):
    if SL_val[i] > SL_val[i-1] and SL_val[i-1]!=0:
        difference.append((SL_val[i]-SL_val[i-1])/SL_val[i-1])

In [ ]:
pd.DataFrame(difference).describe()

Temporary Portion above will be deleted Later

In [ ]:
with open("C:/Users/Harleen.Kaur/Downloads/Spec Object", "r") as file:
    json_data = file.read()
data = json.loads(json_data)    

In [ ]:
data["additional_info"]['dpf_soot_loading_pid']

In [ ]:
vehicle_ID = 1281385415655292928
Start_TS = str(int(1714454580000))
End_TS   = str(int(1716661740000))


local_filename = 'D:/products/EBT_VEHICLE_DATA_FOR_TESTING/'  + str(vehicle_ID) + "_" + str(Start_TS) + "_" + str(End_TS)

# for Indian vehicles use the API 'http://data-download.intangles.com:1883/download/'
# for US vehicles use the API http://algo-internal-apis.intangles-aws-us-east-1.intangles.us:1883/download/
URL = 'https://old-data-downloader.intangles-aws-us-east-1.intangles.us/download/' + str(vehicle_ID) +  "/" + str(Start_TS) + "/" + str(End_TS)



print(URL)

r = requests.get(URL,stream=True)

print("this is the response: ", r)

with open(local_filename, 'wb') as f:
    for chunk in r.iter_content(chunk_size=1024):
        if chunk:
            print("WRITING...........")
            f.write(chunk)
    

In [ ]:
import numpy as np
from matplotlib import pyplot as plt 
import os
import json
import requests
import datetime

# C:\Users\Harleen.Kaur\Downloads\Get_Thresholds.py

def find_nearest(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return array[idx],idx

def extract_PID_data(data, PROTOCOL,LABEL):
    
    if (PROTOCOL == 'SAE'):

        if LABEL == 'IMAP':
            PID_TAG = 'spn_106_avg'
        elif LABEL == 'ENGINE LOAD':
            PID_TAG = 'spn_92_avg'
        elif LABEL == 'ENGINE RPM':
            PID_TAG = 'spn_190_avg'
        elif LABEL == 'FUEL RATE':
            PID_TAG = 'spn_183_avg'
        elif LABEL == 'MAF':
            PID_TAG = 'spn_132_avg'
        elif LABEL == 'BOOST':
            PID_TAG = 'spn_102_avg'
        elif LABEL == 'BAROMETER':
            PID_TAG = 'spn_108_avg'
        elif LABEL == 'THROTTLE':
            PID_TAG = 'spn_91_avg'
        elif LABEL == 'ATGMF': # Eaxuast Gas Flow Rate
            PID_TAG = 'spn_3236_avg'
        elif LABEL == 'DPFDP': # DPF Diffrential Pressure (across DPF)
            PID_TAG = 'spn_3251_avg'
        elif LABEL == 'SCRT':   # SCR Catalyst Temperature Before Catalyst (DPF out)
            PID_TAG = 'spn_4360_avg'
        elif LABEL == 'SPEED': # Wheep based Vehicle Speed
            PID_TAG = 'spn_84_avg'
        elif LABEL == 'DPFINT':# DPF in Temperature Before DPF (DOC out)
            PID_TAG = 'spn_3250_avg' #4766
        elif LABEL == 'IS': #  regen inhibited
            PID_TAG = 'spn_3703_avg'        
        elif LABEL == 'FUEL USE': #  Total fuel used (high precision)
            PID_TAG = 'spn_5054_avg'        
        elif LABEL == 'DISTANCE': #  Total distance travelled (high precision)
            PID_TAG = 'spn_917_avg' # 245
        elif LABEL == 'SOOTLOAD':
            PID_TAG = 'spn_5466_avg' # 245 #3719 generic check       
        elif LABEL == 'ACTIVEREGEN':
            PID_TAG = 'spn_3700_avg' # 245
        elif LABEL == 'PEDAL':
            PID_TAG = 'spn_91_avg' # 245  
        elif LABEL == 'LOAD':
            PID_TAG = 'spn_92_avg' # 245      

    elif(PROTOCOL == 'ISO'):

        if LABEL == 'IMAP':
            PID_TAG = '87BC'#MODIFY the "if PID_TAG in State1:" loop (append for loop on top)  
        elif LABEL == 'ENGINE LOAD':
            PID_TAG = '04'
        elif LABEL == 'ENGINE RPM':
            PID_TAG = '0C'
        elif LABEL == 'FUEL RATE':
            PID_TAG = '5E'
        elif LABEL == 'MAF':
            PID_TAG = '10'
        elif LABEL == 'BOOST':
            PID_TAG = '102'
        elif LABEL == 'BAROMETER':
            PID_TAG = '33'
        elif LABEL == 'THROTTLE':
            PID_TAG = '49'#MODIFY the "if PID_TAG in State1:" loop (append for loop on top)
        elif LABEL == 'DPFDP': # DPF Diffrential Pressure (across DPF)
            PID_TAG = '7A'
        elif LABEL == 'DPFDP': # DPF Diffrential Pressure (across DPF)
            PID_TAG = '7A'    

    #print(LABEL)
    print("PID TAG is-->" + LABEL + " ," +PROTOCOL,"Mapping is --->", PID_TAG)

   

    Time_vec = []
    Val_vec = []
    #print(len(data[0]))
    #print(len(data))

    for data_cnt in range(0,len(data)):
        if "pids" in data[data_cnt]:
            if len(data[data_cnt]['pids'])>0:
                for sub_pid_cnt in range(0,len(data[data_cnt]['pids'])):  #this loop
                    State = data[data_cnt]['pids'][sub_pid_cnt]
                    #print(State)
                    #print(data_cnt)
                    #for state_cnt in range(0,len(State)):
                    if PID_TAG in State:
                        #print("--------------------------------------------IN----------------------------------")
                        Time_vec.append(np.array(State[PID_TAG]['timestamp'], dtype=np.int64))
                        Val_vec.append(np.array(State[PID_TAG]['value'], dtype=float))

    print("Number of time stamp avilables are--->",len(Val_vec))

    
    return Time_vec,Val_vec

In [ ]:
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
import json
import requests
import datetime



if __name__ == '__main__':
    # ... (rest of the code remains the same)
    vehicle =  1281385415655292928
    #imei = 866834049482107 
    # reading the OBD-Data one hour prior and one-hour after the regeneration time
    Start_TS =  1714501800000  - 1*60*60*1000
    End_TS = 1716661740000 + 1*60*60*1000

    OBD_data_path = 'https://old-data-downloader.intangles-aws-us-east-1.intangles.us/download/' + str(vehicle) +  "/" + str(Start_TS) + "/" + str(End_TS)
    r = requests.get(OBD_data_path, stream=True)
    OBD_data = r.json()

    fig = go.Figure()

    LABEL = 'DPFDP'
    X_Time, X_Value = extract_PID_data(OBD_data, 'SAE', LABEL)
    TS = [datetime.datetime.fromtimestamp((X_Time[cnt] + 19800*1000) / 1000.0, tz=datetime.timezone.utc) for cnt in range(0, len(X_Time))]
    fig.add_trace(go.Scatter(x=TS, y=X_Value, mode='markers', name=LABEL))
    fig.add_hline(y=7, line_color='red', line_dash='dash')

    LABEL = 'DPFINT'
    X_Time, X_Value = extract_PID_data(OBD_data, 'SAE', LABEL)
    TS = [datetime.datetime.fromtimestamp((X_Time[cnt] + 19800*1000) / 1000.0, tz=datetime.timezone.utc) for cnt in range(0, len(X_Time))]
    fig.add_trace(go.Scatter(x=TS, y=X_Value, mode='lines', name=LABEL))
    fig.add_hline(y=500, line_color='red', line_dash='dash')

    LABEL = 'ATGMF'
    X_Time, X_Value = extract_PID_data(OBD_data, 'SAE', LABEL)
    TS = [datetime.datetime.fromtimestamp((X_Time[cnt] + 19800*1000) / 1000.0, tz=datetime.timezone.utc) for cnt in range(0, len(X_Time))]
    fig.add_trace(go.Scatter(x=TS, y=X_Value, mode='lines', name=LABEL))

    LABEL = 'ENGINE RPM'
    X_Time, X_Value = extract_PID_data(OBD_data, 'SAE', LABEL)
    TS = [datetime.datetime.fromtimestamp((X_Time[cnt] + 19800*1000) / 1000.0, tz=datetime.timezone.utc) for cnt in range(0, len(X_Time))]
    fig.add_trace(go.Scatter(x=TS, y=X_Value, mode='lines', name=LABEL))

    LABEL = 'THROTTLE'
    X_Time, X_Value = extract_PID_data(OBD_data, 'SAE', LABEL)
    TS = [datetime.datetime.fromtimestamp((X_Time[cnt] + 19800*1000) / 1000.0, tz=datetime.timezone.utc) for cnt in range(0, len(X_Time))]
    fig.add_trace(go.Scatter(x=TS, y=X_Value, mode='lines', name=LABEL))

    LABEL = 'ENGINE LOAD'
    X_Time, X_Value = extract_PID_data(OBD_data, 'SAE', LABEL)
    TS = [datetime.datetime.fromtimestamp((X_Time[cnt] + 19800*1000) / 1000.0, tz=datetime.timezone.utc) for cnt in range(0, len(X_Time))]
    fig.add_trace(go.Scatter(x=TS, y=X_Value, mode='lines', name=LABEL))

    LABEL = 'SPEED'
    X_Time, X_Value = extract_PID_data(OBD_data, 'SAE', LABEL)
    TS = [datetime.datetime.fromtimestamp((X_Time[cnt] + 19800*1000) / 1000.0, tz=datetime.timezone.utc) for cnt in range(0, len(X_Time))]
    fig.add_trace(go.Scatter(x=TS, y=X_Value, mode='lines', name=LABEL))

    LABEL = 'IS'
    X_Time, X_Value = extract_PID_data(OBD_data, 'SAE', LABEL)
    TS = [datetime.datetime.fromtimestamp((X_Time[cnt] + 19800*1000) / 1000.0, tz=datetime.timezone.utc) for cnt in range(0, len(X_Time))]
    fig.add_trace(go.Scatter(x=TS, y=X_Value, mode='lines', name=LABEL))

    LABEL = 'SOOTLOAD'
    X_Time, X_Value = extract_PID_data(OBD_data, 'SAE', LABEL)
    TS = [datetime.datetime.fromtimestamp((X_Time[cnt] + 19800*1000) / 1000.0, tz=datetime.timezone.utc) for cnt in range(0, len(X_Time))]
    fig.add_trace(go.Scatter(x=TS, y=X_Value, mode='lines', name=LABEL))

    LABEL = 'PEDAL'
    X_Time, X_Value = extract_PID_data(OBD_data, 'SAE', LABEL)
    TS = [datetime.datetime.fromtimestamp((X_Time[cnt] + 19800*1000) / 1000.0, tz=datetime.timezone.utc) for cnt in range(0, len(X_Time))]
    fig.add_trace(go.Scatter(x=TS, y=X_Value, mode='lines', name=LABEL))

    LABEL = 'ACTIVEREGEN'
    X_Time, X_Value = extract_PID_data(OBD_data, 'SAE', LABEL)
    TS = [datetime.datetime.fromtimestamp((X_Time[cnt] + 19800*1000) / 1000.0, tz=datetime.timezone.utc) for cnt in range(0, len(X_Time))]
    fig.add_trace(go.Scatter(x=TS, y=X_Value, mode='lines', name=LABEL))

    fig.update_layout(title_text="Vehicle Data", xaxis_title="Time", yaxis_title="Value")
    fig.show()

In [ ]:
dp_pid = 'spn_3251_avg'
rpm_pid = 'spn_190_avg'
speed_pid = 'spn_84_avg'
soot_load_pid = 'spn_5466_avg'
Time = []
dp_inst_Value = []
rpm_inst_Value = []
speed_inst_Value =[]
for pac_idx in range(len(OBD_data)):
        if "pids" in OBD_data[pac_idx]:
            if len(OBD_data[pac_idx]['pids'])>0:
                for sub_pid_cnt in range(0,len(OBD_data[pac_idx]['pids'])):
                        State = OBD_data[pac_idx]['pids'][sub_pid_cnt]
                        # extracting DPFDP
                        if dp_pid in State:
                                Time.append(State[dp_pid]['timestamp'])
                                dp_inst_Value.append(State[dp_pid]['value'][0])
                        # extracting RPM        
                        if rpm_pid in State:
                                rpm_inst_Value.append(State[rpm_pid]['value'][0])
                        # extracting SPEED       
                        if speed_pid in State:
                                speed_inst_Value.append(State[speed_pid]['value'][0])

In [ ]:
import pandas as pd
import numpy as np
import os
import json
#from app.dataIO import get_obd_data 



# function to be called for making the active-regeneration point shift
def active_regeneration_shift(OBD_data, active_regeneration_start_time, spec_obj):
        
        # extracting the relevant soot-load parameter-ID
        soot_load_pid = '5466'
        if 'dpf_soot_loading_pid' in spec_obj["additional_info"]:
               soot_load_pid = spec_obj["additional_info"]['dpf_soot_loading_pid']
        
        Time = []
        soot_load_Value = []
        for pac_idx in range(len(OBD_data)):
                if "pids" in OBD_data[pac_idx]:
                    if len(OBD_data[pac_idx]['pids'])>0:
                        for sub_pid_cnt in range(0,len(OBD_data[pac_idx]['pids'])):
                                State = OBD_data[pac_idx]['pids'][sub_pid_cnt]
                                # extracting Soot-Load
                                if soot_load_pid in State:
                                        Time.append(State[soot_load_pid]['timestamp'])
                                        soot_load_Value.append(State[soot_load_pid]['value'][0])
        
        # if soot_load values not available
        if len(soot_load_Value)==0:
               return active_regeneration_start_time
        
        # if soot load values are available --> find the point where maximum negative slope present
        slope = np.array(soot_load_Value[1:-1]) - np.array(soot_load_Value[0:-2])
        adjusted_index = np.argmin(slope)

        # if the point of maximum drop is greater than 10 minutes
        # search for the maximum drop wthin 10 minutes prior to active-regeneration time
        if (active_regeneration_start_time - Time[adjusted_index-1])>(10*60*1000):
                idx_10_mins_before = (np.abs(np.asarray(Time) - (active_regeneration_start_time - 10*60*1000))).argmin()
                adjusted_index = idx_10_mins_before
        if (Time[adjusted_index-1]-active_regeneration_start_time)>(10*60*1000):
                idx_10_mins_after = (np.abs(np.asarray(Time) - (active_regeneration_start_time + 10*60*1000))).argmin()
                adjusted_index = idx_10_mins_after       
                        

        return Time[adjusted_index-1]
                                      


# function to be called for generation regeneration evidence
def regeneration_evidence(COUNTRY_FLAG, OBD_data,
                            active_regeneration_start_time, active_regeneration_end_time, 
                            burn_quality_percentage):
    
    
                
        # getting the active-regeneration duration in minutes
        active_regeneration_duration = (active_regeneration_end_time - active_regeneration_start_time)//(60*1000)
        
        # corner case
        # if active_regeneration_start_time >= active_regeneration_end_time 
        # active_regeneration_duration happens to be less that 10 minutes
        if (active_regeneration_start_time >= active_regeneration_end_time) or (active_regeneration_duration <= 10):
                # check the status of burn_quality
                # if burn_quality == high  --> Set the speed status sufficient (1)
                # else --> Set the speed status insufficient (0)
                if burn_quality_percentage>=0.6 and burn_quality_percentage!=2:
                        speed_status = 1
                else:
                        speed_status = 0        
                return speed_status, burn_quality_percentage

        
        
        # if FLAG is set to "US" use the following RPM and Speed thresholds
        if COUNTRY_FLAG == 'US':
                RPM_RANGE = [800, 2000]
                SPEED_THRESHOLD = 80

        # if FLAG is set to "IN" use the following RPM and Speed thresholds
        elif COUNTRY_FLAG == 'IN':
                RPM_RANGE = [1000, 2000]
                SPEED_THRESHOLD = 40
 

        # extracting the relevant parameter-ID
        dp_pid = 'spn_3251_avg'
        rpm_pid = 'spn_190_avg'
        speed_pid = 'spn_84_avg'
        Time = []
        dp_inst_Value = []
        rpm_inst_Value = []
        speed_inst_Value =[]
        for pac_idx in range(len(OBD_data)):
                if "pids" in OBD_data[pac_idx]:
                    if len(OBD_data[pac_idx]['pids'])>0:
                        for sub_pid_cnt in range(0,len(OBD_data[pac_idx]['pids'])):
                                State = OBD_data[pac_idx]['pids'][sub_pid_cnt]
                                # extracting DPFDP
                                if dp_pid in State:
                                        Time.append(State[dp_pid]['timestamp'])
                                        dp_inst_Value.append(State[dp_pid]['value'][0])
                                # extracting RPM        
                                if rpm_pid in State:
                                        rpm_inst_Value.append(State[rpm_pid]['value'][0])
                                # extracting SPEED       
                                if speed_pid in State:
                                        speed_inst_Value.append(State[speed_pid]['value'][0])    

                                             
                                
        # applying the RPM constraint
        # getting the correspnding values of different varaibles after applying constraints
        # Time1 constitutes of timestamps where the RPM conditions are met
        dp_rpm_constrained = []
        speed_rpm_constrained = []
        Time1 = []
        rpm_constrained = []

        # iterating through each timestamp to make the RPM constraint check
        for i in range(len(Time)):
            if rpm_inst_Value[i]>=RPM_RANGE[0] and rpm_inst_Value[i]<=RPM_RANGE[-1]:
                    dp_rpm_constrained.append(dp_inst_Value[i])
                    Time1.append(Time[i])
                    rpm_constrained.append(rpm_inst_Value[i])
                    speed_rpm_constrained.append(speed_inst_Value[i])
        
        # after applying the rpm-constraint check which are the nearest active regeneration start and end indices
        nearest_ar_start_idx = (np.abs(np.array(Time1) -  active_regeneration_start_time)).argmin()
        nearest_ar_end_idx = (np.abs(np.array(Time1) -  active_regeneration_end_time)).argmin()
        
        # if the nearest start and end indices comes out to be same (which implies after applying the rpm-constraint we are 
        # not left with any valid point)
        if (nearest_ar_start_idx == nearest_ar_end_idx):
                if burn_quality_percentage>=0.6 and burn_quality_percentage!=2:
                        speed_status = 1
                else:
                        speed_status = 0
                return speed_status, burn_quality_percentage
                


        # calculating the mid-region of the active regeneration zone
        mid_idx = (nearest_ar_start_idx + nearest_ar_end_idx)//2

        # bringing both the pre and post window to same sizes
        pre_start_idx = 0
        post_end_idx = len(Time1)
        # if length of the pre-regeneration window > length of the post-regeneration window
        # bring the pre-window to same size as the post window
        if (mid_idx) > (len(Time1)- nearest_ar_end_idx):
                pre_start_idx = mid_idx - (len(Time1)- nearest_ar_end_idx)
        # if length of the pre-regeneration window < length of the post-regeneration window  
        # bring the post-window to same size as the pre window
        elif (mid_idx) < (len(Time1)- nearest_ar_end_idx):
                post_end_idx = nearest_ar_end_idx + mid_idx 
        
        # fixing the pre_dp, pre_rpm and the pre_speed window
        pre_dp_window = dp_rpm_constrained[pre_start_idx:mid_idx]
        pre_rpm_window = rpm_constrained[pre_start_idx:mid_idx]
        pre_speed_window = speed_rpm_constrained[pre_start_idx:mid_idx]

        # fixing the post_dp, post_rpm and the post_speed window
        post_dp_window = dp_rpm_constrained[nearest_ar_end_idx:post_end_idx]
        post_rpm_window = rpm_constrained[nearest_ar_end_idx:post_end_idx]
        post_speed_window = speed_rpm_constrained[nearest_ar_end_idx:post_end_idx]
        
        # quantifying the burn_quality basis the burn_quality_percentage
        #   0 <= burn_quality_percentage < 0.33 --> low burn_quality
        if (burn_quality_percentage>=0 and burn_quality_percentage<0.33):
                burn_quality = 'low'
        #   0.33 <= burn_quality_percentage < 0.66 --> medium burn_quality        
        elif (burn_quality_percentage>=0.33 and burn_quality_percentage<0.66):
                burn_quality = 'medium' 
        #   0.66 <= burn_quality_percentage < 1 --> high burn_quality        
        elif (burn_quality_percentage>=0.66 and burn_quality_percentage<1):
                burn_quality = 'high' 
        #  burn_quality_percentage == 2 --> failed burn            
        elif burn_quality_percentage == 2:
                burn_quality = 'failed'

        # reconcilation logic for low and high burn-quality        
        if (burn_quality == 'low') or (burn_quality == 'high'):
            
            # getting the partition for breaking the rpm-bins into two equal bins 
            bin_size = (RPM_RANGE[-1]-RPM_RANGE[0])//2       
            # getting the columns (we are capturing the count and mean of the dp values in each of the rpm-bins)
            columns = [
                    'RPM Bin', 'Speed Bin', 'Count', 
                    'Mean'
                    ]

            # capturing the decriptive statistics, and their corresponding rpm and speed bin
            df_pre_statistics_all_vehicles = pd.DataFrame(columns = columns)
            df_post_statistics_all_vehicles = pd.DataFrame(columns = columns)

            pre_descriptive_statistics_all_bins = []
            post_descriptive_statistics_all_bins = []
            pre_rpm_bin_considered = []
            post_rpm_bin_considered = []
            pre_speed_bin_considered = []
            post_speed_bin_considered = []

            # getting the RPM_RANGE and creating bin on those
            for bin in range(RPM_RANGE[0],  RPM_RANGE[-1], bin_size):
                    # creating the low and high speed bins for each of the rpm-bins
                    # getting the pre-bins
                    pre_dp_in_rpm_low_speed_bin = []
                    pre_dp_in_rpm_high_speed_bin = []
                    # getting the post-bins
                    post_dp_in_rpm_low_speed_bin = []
                    post_dp_in_rpm_high_speed_bin = []


                    # parsing through all the enteries of the dp-window
                    for j in range(len(pre_dp_window)):  
                            
                            # if the rpm values are between the bin & (bin+bin_size)

                            # checking the above condition for pre-rpm window
                            if pre_rpm_window[j]>= bin and pre_rpm_window[j] < (bin+bin_size):
                                    # getting the dp in the low-speed zone of specific rpm-zone
                                    if int(pre_speed_window[j]) <= SPEED_THRESHOLD:
                                            pre_dp_in_rpm_low_speed_bin.append(pre_dp_window[j])
                                    # getting the dp in the high-speed zone of specific rpm-zone    
                                    else:
                                            pre_dp_in_rpm_high_speed_bin.append(pre_dp_window[j])   
                            
                            # checking the above condition for the post-window
                            if post_rpm_window[j]>= bin and post_rpm_window[j] < (bin + bin_size):  
                                    # getting the dp in the low-speed zone of specific rpm-zone
                                    if int(post_speed_window[j]) <= SPEED_THRESHOLD:
                                            post_dp_in_rpm_low_speed_bin.append(post_dp_window[j])
                                    # getting the dp in the high-speed zone of specific rpm-zone    
                                    else:
                                            post_dp_in_rpm_high_speed_bin.append(post_dp_window[j])             

                    # if rpm with lower-speed bin
                    if len(pre_dp_in_rpm_low_speed_bin) > 0:
                            summary_statistics =  pd.DataFrame(pre_dp_in_rpm_low_speed_bin).describe()  
                            pre_descriptive_statistics_all_bins.append([
                                                            summary_statistics.loc['count'][0],
                                                            summary_statistics.loc['mean'][0], 
                                                            ]) 
                            pre_rpm_bin_considered.append((bin, (bin+bin_size)))
                            pre_speed_bin_considered.append("<="+str(SPEED_THRESHOLD))
                                            
                    if len(post_dp_in_rpm_low_speed_bin) > 0:
                            summary_statistics =  pd.DataFrame(post_dp_in_rpm_low_speed_bin).describe()  
                            post_descriptive_statistics_all_bins.append([
                                                            summary_statistics.loc['count'][0],
                                                            summary_statistics.loc['mean'][0] 
                                                    ]) 
                            post_rpm_bin_considered.append((bin, (bin+bin_size)))
                            post_speed_bin_considered.append("<="+str(SPEED_THRESHOLD))


                    # if rpm with higher-speed bin                
                    if len(pre_dp_in_rpm_high_speed_bin) > 0:
                            summary_statistics =  pd.DataFrame(pre_dp_in_rpm_high_speed_bin).describe()  
                            pre_descriptive_statistics_all_bins.append([
                                                            summary_statistics.loc['count'][0],
                                                            summary_statistics.loc['mean'][0]]) 
                            pre_rpm_bin_considered.append((bin, (bin+bin_size)))
                            pre_speed_bin_considered.append(">"+str(SPEED_THRESHOLD))

                    if len(post_dp_in_rpm_high_speed_bin) > 0:
                            summary_statistics =  pd.DataFrame(post_dp_in_rpm_high_speed_bin).describe()  
                            post_descriptive_statistics_all_bins.append([
                                                                    summary_statistics.loc['count'][0],
                                                                    summary_statistics.loc['mean'][0]]) 
                            post_rpm_bin_considered.append((bin, (bin+bin_size)))
                            post_speed_bin_considered.append(">"+str(SPEED_THRESHOLD))   
                                                            
                                    
            if len(pre_descriptive_statistics_all_bins):
                for j in range(len(pre_descriptive_statistics_all_bins)):  
                        pre_row_data = []
                        pre_row_data.append(pre_rpm_bin_considered[j]) #'RPM Bin'
                        pre_row_data.append(pre_speed_bin_considered[j]) #'Speed Bin'
                        pre_row_data.extend(pre_descriptive_statistics_all_bins[j]) 
                        df_pre_statistics_all_vehicles.loc[len(df_pre_statistics_all_vehicles)] = pre_row_data
        
            if len(post_descriptive_statistics_all_bins):
                for j in range(len(post_descriptive_statistics_all_bins)):
                        post_row_data = []  
                        post_row_data.append(post_rpm_bin_considered[j]) #'RPM Bin'
                        post_row_data.append(post_speed_bin_considered[j]) #'Speed Bin'
                        post_row_data.extend(post_descriptive_statistics_all_bins[j]) 
                        df_post_statistics_all_vehicles.loc[len(df_post_statistics_all_vehicles)] = post_row_data

            # dropping off duplicates from pre and post dataframe
            df_pre_statistics_all_vehicles = df_pre_statistics_all_vehicles.drop_duplicates()
            df_post_statistics_all_vehicles = df_post_statistics_all_vehicles.drop_duplicates()

            print("this is the pre-statistics: ")
            print(df_pre_statistics_all_vehicles)

            print("this is the pre-statistics: ")
            print(df_post_statistics_all_vehicles)

            # renaming the columns for pre and post statistical dataframes
            df_pre_statistics_all_vehicles.columns = ["Pre RPM Bin", "Pre Speed Bin", 
                                                    "Pre Count", "Pre Mean"]
                    
            df_post_statistics_all_vehicles.columns = ["Post RPM Bin", "Post Speed Bin", 
                                                    "Post Count", "Post Mean"]
                    
            print("this is the pre-statistics: ", df_pre_statistics_all_vehicles)

            print("this is the post-statistics: ", df_post_statistics_all_vehicles)
            
            
            # combining pre and post dataframes basis RPM and SPEED bin values 
            df_statistics = pd.merge(df_pre_statistics_all_vehicles, df_post_statistics_all_vehicles, how='inner', 
                                    left_on=["Pre RPM Bin", "Pre Speed Bin"],
                                    right_on = ["Post RPM Bin", "Post Speed Bin"])
                    
            # In case any duplicates generated while combining the two dataframes drop them
            if df_statistics.shape[0]>0:

                df_statistics = df_statistics.drop_duplicates()

                print("this is the df_statistics using the version-1: ", df_statistics) 

                # dropping all those statistical bins wherein the Pre or Post Count of values are less than 5
                indices_remove = df_statistics[(df_statistics['Pre Count']<5) | (df_statistics['Post Count']<5)].index
                df_statistics.drop(indices_remove, inplace=True)

                
                
                fraction_good_bins = df_statistics[(df_statistics["Post Mean"]<df_statistics["Pre Mean"])].shape[0]/df_statistics.shape[0]


                print("these are the fraction of good-bins using the version-1: ", fraction_good_bins)
                
        
                # for "high" soot burn
                # the logic goes something like this
                # if 0/4 bins has the post-dp < pre-dp, classify as low
                # if 1/4 or 2/4 has the post-dp < pre-dp, classify as medium 
                if burn_quality == 'high' and fraction_good_bins == 0:
                        burn_quality = 'low'
                        burn_quality_percentage = burn_quality_percentage/3
                elif burn_quality == 'high' and fraction_good_bins <= 0.5 and fraction_good_bins != 0:
                        burn_quality = 'medium'
                        burn_quality_percentage = burn_quality_percentage/2   

                # for "low" soot burn
                # the logic goes something like this
                # if 3/4 or 4/4 has the post-dp < pre-dp, classify as medium 
                if burn_quality == 'low' and fraction_good_bins > 0.5:
                        burn_quality = 'medium'
                        burn_quality_percentage = burn_quality_percentage * 2
        high_speed_count = 0
        # giving the duration and speed status for each of the moderate and low burn_quality
        if burn_quality != 'high':     
                # if for atleast 50% of the time; the vehicle is running at required speed  
                # then set the speed status to sufficient i.e. 1        
                for speed in speed_inst_Value:
                        if speed >= SPEED_THRESHOLD:
                                high_speed_count += 1
                # if for atleast 50% of the time speed happens to be high during regeneration                
                if high_speed_count/len(speed_inst_Value) > 0.5:    
                        speed_status = 1
                        
                elif  high_speed_count/len(speed_inst_Value) <= 0.5: 
                        speed_status = 0
        # if burn_quality is high ; keep the speed status sufficient               
        elif burn_quality == 'high':
                speed_status = 1  

                        

        # return the speed, duration status and the time for which the function is executed
        return speed_status, burn_quality_percentage 


# function to be called for generation regeneration evidence
def regeneration_evidence_v2(COUNTRY_FLAG, OBD_data,
                            active_regeneration_start_time, active_regeneration_end_time, 
                            burn_quality_percentage):
    
    
                
        # getting the active-regeneration duration in minutes
        active_regeneration_duration = (active_regeneration_end_time - active_regeneration_start_time)//(60*1000)
        
        # corner case
        # if active_regeneration_start_time >= active_regeneration_end_time 
        # active_regeneration_duration happens to be less that 10 minutes
        if (active_regeneration_start_time >= active_regeneration_end_time) or (active_regeneration_duration <= 10):
                # check the status of burn_quality
                # if burn_quality == high  --> Set the speed status sufficient (1)
                # else --> Set the speed status insufficient (0)
                if burn_quality_percentage>=0.6 and burn_quality_percentage!=2:
                        speed_status = 1
                else:
                        speed_status = 0        
                return speed_status, burn_quality_percentage

        
        
        # if FLAG is set to "US" use the following RPM and Speed thresholds
        if  COUNTRY_FLAG == 'US':
                RPM_RANGE = [800, 2000]
                SPEED_THRESHOLD = 80

        # if FLAG is set to "IN" use the following RPM and Speed thresholds
        elif COUNTRY_FLAG == 'IN':
                RPM_RANGE = [1000, 2000]
                SPEED_THRESHOLD = 40
 

        # extracting the relevant parameter-ID
        dp_pid = 'spn_3251_avg'
        rpm_pid = 'spn_190_avg'
        speed_pid = 'spn_84_avg'
        Time = []
        dp_inst_Value = []
        rpm_inst_Value = []
        speed_inst_Value =[]
        for pac_idx in range(len(OBD_data)):
                if "pids" in OBD_data[pac_idx]:
                    if len(OBD_data[pac_idx]['pids'])>0:
                        for sub_pid_cnt in range(0,len(OBD_data[pac_idx]['pids'])):
                                State = OBD_data[pac_idx]['pids'][sub_pid_cnt]
                                # extracting DPFDP
                                if dp_pid in State:
                                        Time.append(State[dp_pid]['timestamp'])
                                        dp_inst_Value.append(State[dp_pid]['value'][0])
                                # extracting RPM        
                                if rpm_pid in State:
                                        rpm_inst_Value.append(State[rpm_pid]['value'][0])
                                # extracting SPEED       
                                if speed_pid in State:
                                        speed_inst_Value.append(State[speed_pid]['value'][0])    

                                             
                                
        # applying the RPM constraint
        # getting the correspnding values of different varaibles after applying constraints
        # Time1 constitutes of timestamps where the RPM conditions are met
        dp_rpm_constrained = []
        speed_rpm_constrained = []
        Time1 = []
        rpm_constrained = []

        # iterating through each timestamp to make the RPM constraint check
        for i in range(len(Time)):
            if rpm_inst_Value[i]>=RPM_RANGE[0] and rpm_inst_Value[i]<=RPM_RANGE[-1]:
                    dp_rpm_constrained.append(dp_inst_Value[i])
                    Time1.append(Time[i])
                    rpm_constrained.append(rpm_inst_Value[i])
                    speed_rpm_constrained.append(speed_inst_Value[i])
        
        # after applying the rpm-constraint check which are the nearest active regeneration start and end indices
        nearest_ar_start_idx = (np.abs(np.array(Time1) -  active_regeneration_start_time)).argmin()
        nearest_ar_end_idx = (np.abs(np.array(Time1) -  active_regeneration_end_time)).argmin()
        
        # if the nearest start and end indices comes out to be same (which implies after applying the rpm-constraint we are 
        # not left with any valid point)
        if (nearest_ar_start_idx == nearest_ar_end_idx):
                if burn_quality_percentage>=0.6 and burn_quality_percentage!=2:
                        speed_status = 1
                else:
                        speed_status = 0
                return speed_status, burn_quality_percentage
                


        # calculating the mid-region of the active regeneration zone
        mid_idx = (nearest_ar_start_idx + nearest_ar_end_idx)//2

        # bringing both the pre and post window to same sizes
        pre_start_idx = 0
        post_end_idx = len(Time1)
        # if length of the pre-regeneration window > length of the post-regeneration window
        # bring the pre-window to same size as the post window
        if (mid_idx) > (len(Time1)- nearest_ar_end_idx):
                pre_start_idx = mid_idx - (len(Time1)- nearest_ar_end_idx)
        # if length of the pre-regeneration window < length of the post-regeneration window  
        # bring the post-window to same size as the pre window
        elif (mid_idx) < (len(Time1)- nearest_ar_end_idx):
                post_end_idx = nearest_ar_end_idx + mid_idx 
        
        # fixing the pre_dp, pre_rpm and the pre_speed window
        pre_dp_window = dp_rpm_constrained[pre_start_idx:mid_idx]
        pre_rpm_window = rpm_constrained[pre_start_idx:mid_idx]
        pre_speed_window = speed_rpm_constrained[pre_start_idx:mid_idx]

        # fixing the post_dp, post_rpm and the post_speed window
        post_dp_window = dp_rpm_constrained[nearest_ar_end_idx:post_end_idx]
        post_rpm_window = rpm_constrained[nearest_ar_end_idx:post_end_idx]
        post_speed_window = speed_rpm_constrained[nearest_ar_end_idx:post_end_idx]
        
        # quantifying the burn_quality basis the burn_quality_percentage
        #   0 <= burn_quality_percentage < 0.33 --> low burn_quality
        if (burn_quality_percentage>=0 and burn_quality_percentage<0.33):
                burn_quality = 'low'
        #   0.33 <= burn_quality_percentage < 0.66 --> medium burn_quality        
        elif (burn_quality_percentage>=0.33 and burn_quality_percentage<0.66):
                burn_quality = 'medium' 
        #   0.66 <= burn_quality_percentage < 1 --> high burn_quality        
        elif (burn_quality_percentage>=0.66 and burn_quality_percentage<1):
                burn_quality = 'high' 
        #  burn_quality_percentage == 2 --> failed burn            
        elif burn_quality_percentage == 2:
                burn_quality = 'failed'

        # reconcilation logic for low and high burn-quality        
        if (burn_quality == 'low') or (burn_quality == 'high'):
            
            # getting the partition for breaking the rpm-bins into two equal bins 
            bin_size = (RPM_RANGE[-1]-RPM_RANGE[0])//2       
            
        
            pre_descriptive_statistics_all_bins = []
            post_descriptive_statistics_all_bins = []
            pre_rpm_bin_considered = []
            post_rpm_bin_considered = []
            pre_speed_bin_considered = []
            post_speed_bin_considered = []

            # getting the RPM_RANGE and creating bin on those
            for bin in range(RPM_RANGE[0],  RPM_RANGE[-1], bin_size):
                    # creating the low and high speed bins for each of the rpm-bins
                    # getting the pre-bins
                    pre_dp_in_rpm_low_speed_bin = []
                    pre_dp_in_rpm_high_speed_bin = []
                    # getting the post-bins
                    post_dp_in_rpm_low_speed_bin = []
                    post_dp_in_rpm_high_speed_bin = []


                    # parsing through all the enteries of the dp-window
                    for j in range(len(pre_dp_window)):  
                            
                            # if the rpm values are between the bin & (bin+bin_size)

                            # checking the above condition for pre-rpm window
                            if pre_rpm_window[j]>= bin and pre_rpm_window[j] < (bin+bin_size):
                                    # getting the dp in the low-speed zone of specific rpm-zone
                                    if int(pre_speed_window[j]) <= SPEED_THRESHOLD:
                                            pre_dp_in_rpm_low_speed_bin.append(pre_dp_window[j])
                                    # getting the dp in the high-speed zone of specific rpm-zone    
                                    else:
                                            pre_dp_in_rpm_high_speed_bin.append(pre_dp_window[j])   
                            
                            # checking the above condition for the post-window
                            if post_rpm_window[j]>= bin and post_rpm_window[j] < (bin + bin_size):  
                                    # getting the dp in the low-speed zone of specific rpm-zone
                                    if int(post_speed_window[j]) <= SPEED_THRESHOLD:
                                            post_dp_in_rpm_low_speed_bin.append(post_dp_window[j])
                                    # getting the dp in the high-speed zone of specific rpm-zone    
                                    else:
                                            post_dp_in_rpm_high_speed_bin.append(post_dp_window[j])             

                    # if rpm with lower-speed bin
                    if len(pre_dp_in_rpm_low_speed_bin) > 0:  
                            pre_descriptive_statistics_all_bins.append([
                                                            len(pre_dp_in_rpm_low_speed_bin),
                                                            np.mean(pre_dp_in_rpm_low_speed_bin) 
                                                            ]) 
                            pre_rpm_bin_considered.append((bin, (bin+bin_size)))
                            pre_speed_bin_considered.append("<="+str(SPEED_THRESHOLD))
                                            
                    if len(post_dp_in_rpm_low_speed_bin) > 0: 
                            post_descriptive_statistics_all_bins.append([
                                                             len(post_dp_in_rpm_low_speed_bin),
                                                             np.mean(post_dp_in_rpm_low_speed_bin) 
                                                    ]) 
                            post_rpm_bin_considered.append((bin, (bin+bin_size)))
                            post_speed_bin_considered.append("<="+str(SPEED_THRESHOLD))


                    # if rpm with higher-speed bin                
                    if len(pre_dp_in_rpm_high_speed_bin) > 0:
                            pre_descriptive_statistics_all_bins.append([
                                                            len(pre_dp_in_rpm_high_speed_bin),
                                                            np.mean(pre_dp_in_rpm_high_speed_bin)])
                            pre_rpm_bin_considered.append((bin, (bin+bin_size)))
                            pre_speed_bin_considered.append(">"+str(SPEED_THRESHOLD))

                    if len(post_dp_in_rpm_high_speed_bin) > 0:
                            post_descriptive_statistics_all_bins.append([
                                                                   len(post_dp_in_rpm_high_speed_bin),
                                                                   np.mean(post_dp_in_rpm_high_speed_bin)]) 
                            post_rpm_bin_considered.append((bin, (bin+bin_size)))
                            post_speed_bin_considered.append(">"+str(SPEED_THRESHOLD))   
                                                            
                                    
            pre_descriptive_statistics_all_bins = np.array(pre_descriptive_statistics_all_bins)
            post_descriptive_statistics_all_bins = np.array(post_descriptive_statistics_all_bins)

            # Create a function to create a 2D array from input arrays
            def create_2d_array(arr1, arr2, arr3):
                return np.column_stack((arr1, arr2, arr3))

            
            if len(pre_descriptive_statistics_all_bins):
                pre_statistics = create_2d_array(pre_rpm_bin_considered, pre_speed_bin_considered, pre_descriptive_statistics_all_bins)
        
            if len(post_descriptive_statistics_all_bins):
                post_statistics = create_2d_array(post_rpm_bin_considered, post_speed_bin_considered, post_descriptive_statistics_all_bins)
                

            # Function to remove duplicates from a 2D array
            def remove_duplicates(arr):
                _, unique_indices = np.unique(arr, axis=0, return_index=True)
                return arr[np.sort(unique_indices)]

            # Remove duplicates from pre and post statistics arrays
            if pre_descriptive_statistics_all_bins.size > 0:
                pre_statistics = remove_duplicates(pre_statistics)

            if post_descriptive_statistics_all_bins.size > 0:
                post_statistics = remove_duplicates(post_statistics)


            # Function to merge pre and post statistics arrays based on RPM and Speed bins
            def merge_statistics(pre_arr, post_arr):
                pre_rpm_bin = pre_arr[:, 0]
                pre_speed_bin = pre_arr[:, 1]
                post_rpm_bin = post_arr[:, 0]
                post_speed_bin = post_arr[:, 1]

                merged_indices = np.where((np.isin(pre_rpm_bin, post_rpm_bin)) & (np.isin(pre_speed_bin, post_speed_bin)))[0]
                merged_pre = pre_arr[merged_indices]
                merged_post = post_arr[merged_indices]

                return np.column_stack((merged_pre, merged_post[:, 2:]))

            # Merge pre and post statistics arrays
            if pre_descriptive_statistics_all_bins.size > 0 and post_descriptive_statistics_all_bins.size > 0:
                df_statistics = merge_statistics(pre_statistics, post_statistics)

            print("this is the df_statistics using the version-2: ", df_statistics)     


            # Filter out bins with counts less than 5 in either Pre or Post
            df_statistics = df_statistics[(df_statistics[:, 2] >= 5) & (df_statistics[:, 4] >= 5)]

            # Calculate fraction of good bins where Post Mean < Pre Mean
            fraction_good_bins = np.sum(df_statistics[:, 3] < df_statistics[:, 5]) / df_statistics.shape[0]


            print("these are the fraction of good-bins using the version-2: ", fraction_good_bins)

    
            # for "high" soot burn
            # the logic goes something like this
            # if 0/4 bins has the post-dp < pre-dp, classify as low
            # if 1/4 or 2/4 has the post-dp < pre-dp, classify as medium 
            if burn_quality == 'high' and fraction_good_bins == 0:
                    burn_quality = 'low'
                    burn_quality_percentage = burn_quality_percentage/3
            elif burn_quality == 'high' and fraction_good_bins <= 0.5 and fraction_good_bins != 0:
                    burn_quality = 'medium'
                    burn_quality_percentage = burn_quality_percentage/2   

            # for "low" soot burn
            # the logic goes something like this
            # if 3/4 or 4/4 has the post-dp < pre-dp, classify as medium 
            if burn_quality == 'low' and fraction_good_bins > 0.5:
                    burn_quality = 'medium'
                    burn_quality_percentage = burn_quality_percentage * 2
        high_speed_count = 0
        # giving the duration and speed status for each of the moderate and low burn_quality
        if burn_quality != 'high':     
                # if for atleast 50% of the time; the vehicle is running at required speed  
                # then set the speed status to sufficient i.e. 1        
                for speed in speed_inst_Value:
                        if speed >= SPEED_THRESHOLD:
                                high_speed_count += 1
                # if for atleast 50% of the time speed happens to be high during regeneration                
                if high_speed_count/len(speed_inst_Value) > 0.5:    
                        speed_status = 1
                        
                elif  high_speed_count/len(speed_inst_Value) <= 0.5: 
                        speed_status = 0
        # if burn_quality is high ; keep the speed status sufficient               
        elif burn_quality == 'high':
                speed_status = 1  

                        

        # return the speed, duration status and the time for which the function is executed
        return speed_status, burn_quality_percentage 



def REGENERATION_EVIDENCE_MSTR(vehicle_id, COUNTRY_FLAG, active_regeneration_start_time, active_regeneration_end_time, 
                            burn_quality_percentage, spec_obj):
    
        # extracting the OBD-data 
        
        Start_TS = active_regeneration_start_time - 60*60*1000
        End_TS = active_regeneration_end_time + 60*60*1000

        #OBD_data = get_obd_data(vehicle_id, Start_TS, End_TS)
        URL = 'https://old-data-downloader.intangles-aws-us-east-1.intangles.us/download/' + str(vehicle_id) +  "/" + str(Start_TS) + "/" + str(End_TS)
        r = requests.get(URL,stream=True)
        OBD_data = r.json()     
   

        
        # if the OBD_data is not empty
        if len(OBD_data): 
                # mapping the actual-regeneration time
                actual_regeneration_time = active_regeneration_shift(OBD_data, active_regeneration_start_time, spec_obj)
                speed_status, burn_quality_percentage = regeneration_evidence(COUNTRY_FLAG, OBD_data,
                                                                active_regeneration_start_time, active_regeneration_end_time, 
                                                                burn_quality_percentage)
        else:
                actual_regeneration_time = active_regeneration_start_time
                # if burn_quality is high --> speed_status should be sufficient --> 1
                if burn_quality_percentage > 0.6:
                        speed_status = 1
                # if burn quality anything apart from high --> speed status should be insufficient --> 0         
                else:
                        speed_status = 0
        

        return speed_status, burn_quality_percentage, actual_regeneration_time 









                

In [ ]:
vehicle_id = 1281011284925480960
active_regeneration_start_time = 1715983200000
active_regeneration_end_time = 1715986800000
COUNTRY_FLAG = 'US'
burn_quality_percentage = 0.2



In [ ]:
file_path = "C:/Users/Harleen.Kaur/Downloads/Spec Object"

# Open the JSON file and load its contents
with open(file_path, "r") as file:
    spec_obj = json.load(file)

In [ ]:
speed_status, burn_quality_percentage, actual_regeneration_time = REGENERATION_EVIDENCE_MSTR(vehicle_id, COUNTRY_FLAG, active_regeneration_start_time, active_regeneration_end_time, 
                            burn_quality_percentage, spec_obj)

In [ ]:
speed_status

In [ ]:
burn_quality_percentage

In [ ]:
vehicle_id = 1281011284925480960
Start_TS = 1715944320000 
End_TS = 1716143340000
URL = 'https://old-data-downloader.intangles-aws-us-east-1.intangles.us/download/' + str(vehicle_id) +  "/" + str(Start_TS) + "/" + str(End_TS)
r = requests.get(URL,stream=True)
print(r)
OBD_data = r.json()

In [ ]:
dp_pid = 'spn_3251_avg'
rpm_pid = 'spn_190_avg'
speed_pid = 'spn_84_avg'
Time = []
dp_inst_Value = []
rpm_inst_Value = []
speed_inst_Value =[]
for pac_idx in range(len(OBD_data)):
        if "pids" in OBD_data[pac_idx]:
            if len(OBD_data[pac_idx]['pids'])>0:
                for sub_pid_cnt in range(0,len(OBD_data[pac_idx]['pids'])):
                        State = OBD_data[pac_idx]['pids'][sub_pid_cnt]
                        # extracting DPFDP
                        if dp_pid in State:
                                Time.append(State[dp_pid]['timestamp'])
                                dp_inst_Value.append(State[dp_pid]['value'][0])
                        # extracting RPM        
                        if rpm_pid in State:
                                rpm_inst_Value.append(State[rpm_pid]['value'][0])
                        # extracting SPEED       
                        if speed_pid in State:
                                speed_inst_Value.append(State[speed_pid]['value'][0])

In [ ]:
dp_inst_Value

In [ ]:
def find_nearest(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return array[idx],idx

In [ ]:
def RPM_Contraint(DATA,T1,RANGE):

    TEMP = DATA[0,:] # 0th is RPM
    nums = TEMP[(RANGE[0] < TEMP) & (TEMP <= RANGE[1])]
    Samples = len(nums)
    #print(Samples)
    DATA_OUT = np.zeros((DATA.shape[0],Samples))
    T_OUT = np.zeros(Samples)

    out_cnt = 0

    for cnt in range(0,DATA.shape[1]):
        if ((RANGE[0] < TEMP[cnt]) and (TEMP[cnt] <= RANGE[1])):
            for var_cnt in range(0,DATA.shape[0]):
                DATA_OUT[var_cnt,out_cnt] = DATA[var_cnt,cnt]
            T_OUT[out_cnt] = T1[cnt]
            out_cnt = out_cnt + 1

    return DATA_OUT,T_OUT
    

def Throttle_Contraint(DATA,T2,TH):

    TEMP = DATA[2,:] # 7th is Throttle
    nums = TEMP[(TH < TEMP)]
    Samples = len(nums)
    #print(Samples)
    DATA_OUT = np.zeros((DATA.shape[0],Samples))
    T_OUT = np.zeros(Samples)

    out_cnt = 0

    for cnt in range(0,DATA.shape[1]):
        if (TH < TEMP[cnt]):
            for var_cnt in range(0,DATA.shape[0]):
                DATA_OUT[var_cnt,out_cnt] = DATA[var_cnt,cnt]
            T_OUT[out_cnt] = T2[cnt]
            out_cnt = out_cnt + 1

    return DATA_OUT,T_OUT

In [ ]:
def extract_PID_data_v3(data, PROTOCOL,LABEL):
    
    if (PROTOCOL == 'SAE'):

        if LABEL == 'IMAP':
            PID_TAG = '106'
        elif LABEL == 'ENGINE LOAD':
            PID_TAG = '92'
        elif LABEL == 'ENGINE RPM':
            PID_TAG = '190'
        elif LABEL == 'FUEL RATE':
            PID_TAG = '183'
        elif LABEL == 'MAF':
            PID_TAG = '132'
        elif LABEL == 'BOOST':
            PID_TAG = '102'
        elif LABEL == 'BAROMETER':
            PID_TAG = '108'
        elif LABEL == 'THROTTLE':
            PID_TAG = '91'
        elif LABEL == 'ATGMF': # Eaxuast Gas Flow Rate
            PID_TAG = '3236'
        elif LABEL == 'DPFDP': # DPF Diffrential Pressure (across DPF)
            PID_TAG = '3251'
        elif LABEL == 'SCRT':   # SCR Catalyst Temperature Before Catalyst (DPF out)
            PID_TAG = '4360'
        elif LABEL == 'SPEED': # Wheep based Vehicle Speed
            PID_TAG = '84'
        elif LABEL == 'DPFINT':# DPF in Temperature Before DPF (DOC out)
            PID_TAG = '4766' #4766
        elif LABEL == 'IS': #  regen inhibited
            PID_TAG = '3703'        
        elif LABEL == 'FUEL USE': #  Total fuel used (high precision)
            PID_TAG = '5054'        
        elif LABEL == 'DISTANCE': #  Total distance travelled (high precision)
            PID_TAG = '917' # 245
        elif LABEL == 'SOOTLOAD':
            PID_TAG = '5466' # 245 #3719 generic check       
        elif LABEL == 'ACTIVEREGEN':
            PID_TAG = '3700' # 245

    elif (PROTOCOL == 'SAE_AVG'):

        if LABEL == 'IMAP':
            PID_TAG = 'spn_106_avg'
        elif LABEL == 'ENGINE LOAD':
            PID_TAG = 'spn_92_avg'
        elif LABEL == 'ENGINE RPM':
            PID_TAG = 'spn_190_avg'
        elif LABEL == 'FUEL RATE':
            PID_TAG = 'spn_183_avg'
        elif LABEL == 'MAF':
            PID_TAG = 'spn_132_avg'
        elif LABEL == 'BOOST':
            PID_TAG = 'spn_102_avg'
        elif LABEL == 'BAROMETER':
            PID_TAG = 'spn_108_avg'
        elif LABEL == 'THROTTLE':
            PID_TAG = 'spn_91_avg'
        elif LABEL == 'ATGMF': # Eaxuast Gas Flow Rate
            PID_TAG = 'spn_3236_avg'
        elif LABEL == 'DPFDP': # DPF Diffrential Pressure (across DPF)
            PID_TAG = 'spn_3251_avg'
        elif LABEL == 'SCRT':   # SCR Catalyst Temperature Before Catalyst (DPF out)
            PID_TAG = 'spn_4360_avg'
        elif LABEL == 'SPEED': # Wheep based Vehicle Speed
            PID_TAG = 'spn_84_avg'
        elif LABEL == 'DPFINT':# DPF in Temperature Before DPF (DOC out)
            PID_TAG = 'spn_4766_avg' #4766
        elif LABEL == 'IS': #  regen inhibited
            PID_TAG = 'spn_3703_avg'        
        elif LABEL == 'FUEL USE': #  Total fuel used (high precision)
            PID_TAG = 'spn_5054_avg'        
        elif LABEL == 'DISTANCE': #  Total distance travelled (high precision)
            PID_TAG = 'spn_917_avg' # 245
        elif LABEL == 'SOOTLOAD':
            PID_TAG = 'spn_5466_avg' # 245 #3719 generic check       
        elif LABEL == 'ACTIVEREGEN':
            PID_TAG = 'spn_3700_avg' # 245
        

    elif(PROTOCOL == 'ISO'):

        if LABEL == 'IMAP':
            PID_TAG = '87BC'#MODIFY the "if PID_TAG in State1:" loop (append for loop on top)  
        elif LABEL == 'ENGINE LOAD':
            PID_TAG = '04'
        elif LABEL == 'ENGINE RPM':
            PID_TAG = '0C'
        elif LABEL == 'FUEL RATE':
            PID_TAG = '5E'
        elif LABEL == 'MAF':
            PID_TAG = '10'
        elif LABEL == 'BOOST':
            PID_TAG = '102'
        elif LABEL == 'BAROMETER':
            PID_TAG = '33'
        elif LABEL == 'THROTTLE':
            PID_TAG = '49'#MODIFY the "if PID_TAG in State1:" loop (append for loop on top)
        elif LABEL == 'DPFDP': # DPF Diffrential Pressure (across DPF)
            PID_TAG = '7A'
    Time_vec = []
    Val_vec = []


    for i in range(len(data)):
        if "pids" in data[i]:
            if len(data[i]['pids'])>0:
                for sub_pid_cnt in range(0,len(data[i]['pids'])):
                    State = data[i]['pids'][sub_pid_cnt]
                    if PID_TAG in State:
                        Time_vec.append(State[PID_TAG]['timestamp'])
                        Val_vec.append(State[PID_TAG]['value'][0])
      
                
    return Time_vec,Val_vec        
            

In [ ]:
def regeneration_evidence_v3(vehicle_id, FLAG,
                            active_regeneration_start_time, active_regeneration_end_time, 
                            burn_quality_percentage, Median_Duration):
    
    
        if active_regeneration_start_time >= active_regeneration_end_time:
            duration_status = "Invalid Start and End Time"
            speed_status ="NA"
            return speed_status, duration_status, burn_quality_percentage

        # getting the active-regeneration duration
        active_regeneration_duration = (active_regeneration_end_time - active_regeneration_start_time)//(60*1000)

        print("this is the active-Regeneration Duration: ", active_regeneration_duration)

        # if the duration of active-regeneration is less than 10 minutes 
        if active_regeneration_duration <= 10:
                        duration_status = "insufficient"
                        speed_status ="NA"
                        return speed_status, duration_status, burn_quality_percentage
        
        # defining the intial and final time stamps needed for creating pre and post regeneration windows
        # We are selecting the time which is 1 hour before the start of the active-regeneration event
        # and 1 hour after the ative-regeneration event
        Start_TS = active_regeneration_start_time - 1*60*60*1000
        End_TS = active_regeneration_end_time + 1*60*60*1000

       
        
        # if FLAG is set to "US", use the US URL link
        if FLAG == 'US':
                
                RPM_RANGE = [800, 2000]
                SPEED_THRESHOLD = 80

                # loading the OBD_data
                URL = 'https://old-data-downloader.intangles-aws-us-east-1.intangles.us/download/' + str(vehicle_id) +  "/" + str(Start_TS) + "/" + str(End_TS)
                
        # if FLAG is set to "IN", use the Indian URL link
        elif FLAG == 'IN':
                
                RPM_RANGE = [1000, 2000]
                SPEED_THRESHOLD = 40

                # loading the OBD_data
                URL = 'http://data-download.intangles.com:1883/download/' + str(vehicle_id) +  "/" + str(Start_TS) + "/" + str(End_TS)
                

        # getting the OBD data between start and end period
        r = requests.get(URL,stream=True)
        OBD_data = r.json()            
        
        # extracting the relevant parameter-ID
        # extracting the variable Differential Pressure
        Time, dp_inst_Value = extract_PID_data_v3(OBD_data, 'SAE_AVG', 'DPFDP')
        # extracting the varible Engine RPM
        Time, rpm_inst_Value = extract_PID_data_v3(OBD_data, 'SAE_AVG', 'ENGINE RPM')
        # extracting the variable SPEED
        Time, speed_inst_Value = extract_PID_data_v3(OBD_data, 'SAE_AVG', 'SPEED')


        print(Time)
        
       

        # applying the RPM constraint
        # getting the correspnding values of different varaibles after applying constraint
        # Time1 constitutes of timestamps where the RPM conditions are met
        dp_rpm_constrained = []
        speed_rpm_constrained = []
        Time1 = []
        rpm_constrained = []

        # iterating through each timestamp to make the RPM constraint check
        for i in range(len(Time)):
                if rpm_inst_Value[i]>=RPM_RANGE[0] and rpm_inst_Value[i]<=RPM_RANGE[-1]:
                        dp_rpm_constrained.append(dp_inst_Value[i])
                        Time1.append(Time[i])
                        rpm_constrained.append(rpm_inst_Value[i])
                        speed_rpm_constrained.append(speed_inst_Value[i])
        
        # after applying the rpm-constraint check which are the nearest active regeneration start and end indices
        nearest_ar_start_idx = (np.abs(np.array(Time1) -  active_regeneration_start_time)).argmin()
        nearest_ar_end_idx = np.abs(np.array(Time1) -  active_regeneration_end_time).argmin()
        
        # if the nearest start and end indices comes out to be same (which implies after applying the rpm-constrint we are not left with any valid variable points)
        if (nearest_ar_start_idx == nearest_ar_end_idx):
                duration_status = "insufficient"
                speed_status ="NA"
                return speed_status, duration_status, burn_quality_percentage
                


        # calculating the mid-region of the active regeneration zone
        mid_idx = (nearest_ar_start_idx + nearest_ar_end_idx)//2
       
        # bringing both the pre and post window to same sizes
        pre_start_idx = 0
        post_end_idx = len(Time1)
        # if length of the pre-regeneration window > length of the post-regeneration window
        # bring the pre-window to same size as the post window
        if (mid_idx) > (len(Time1)- nearest_ar_end_idx):
            pre_start_idx = mid_idx - (len(Time1)- nearest_ar_end_idx)
        # if length of the pre-regeneration window < length of the post-regeneration window  
        # bring the post-window to same size as the pre window
        elif   (mid_idx) < (len(Time1)- nearest_ar_end_idx):
                post_end_idx = nearest_ar_end_idx + mid_idx  


        # fixing the pre_dp; pre_rpm and the pre_speed window
        pre_dp_window = dp_rpm_constrained[pre_start_idx:mid_idx]
        pre_rpm_window = rpm_constrained[pre_start_idx:mid_idx]
        pre_speed_window = speed_rpm_constrained[pre_start_idx:mid_idx]

        # fixing the post_dp; post_rpm and the post_speed window
        post_dp_window = dp_rpm_constrained[nearest_ar_end_idx:post_end_idx]
        post_rpm_window = rpm_constrained[nearest_ar_end_idx:post_end_idx]
        post_speed_window = speed_rpm_constrained[nearest_ar_end_idx:post_end_idx]

        
        # quantifiying the burn_quality basis the burn_quality_percentage
        #   0 <= burn_quality_percentage < 0.33 --> low burn_quality
        if (burn_quality_percentage>=0 and burn_quality_percentage<0.33):
                burn_quality = 'low'
        #   0.33 <= burn_quality_percentage < 0.66 --> medium burn_quality        
        elif (burn_quality_percentage>=0.33 and burn_quality_percentage<0.66):
                burn_quality = 'medium' 
        #   0.66 <= burn_quality_percentage < 1 --> high burn_quality        
        elif (burn_quality_percentage>=0.66 and burn_quality_percentage<1):
                burn_quality = 'high' 
         #  burn_quality_percentage == 2 --> failed burn            
        elif burn_quality_percentage == 2:
                burn_quality = 'failed'

        # reconcilation logic for low and high burn-quality        
        if (burn_quality == 'low') or (burn_quality == 'high'):
                        
                        
                        # getting the partition for breaking the rpm-bins into two euqla bins 
                        bin_size = (RPM_RANGE[-1]-RPM_RANGE[0])//2       
                        # getting the columns (we area capturing the count and mean of the dp values in ecah of the rpm-bins)
                        columns = [
                                'RPM Bin', 'Speed Bin', 'Count', 
                                'Mean'
                                ]

                        # capturing the decriptive statistics, and their corresponding rpm and speed bin
                        df_pre_statistics_all_vehicles = pd.DataFrame(columns = columns)
                        df_post_statistics_all_vehicles = pd.DataFrame(columns = columns)

                        pre_descriptive_statistics_all_bins = []
                        post_descriptive_statistics_all_bins = []
                        pre_rpm_bin_considered = []
                        post_rpm_bin_considered = []
                        pre_speed_bin_considered = []
                        post_speed_bin_considered = []

                        
                        print("this is the RPM Range: ", RPM_RANGE)

                        # getting the RPM_RANGE and creating bin-on those
                        for bin in range(RPM_RANGE[0],  RPM_RANGE[-1], bin_size):
                                # creating the low and high speed bins for each of the rpm-bins
                                # getting the pre-bins
                                pre_dp_in_rpm_low_speed_bin = []
                                pre_dp_in_rpm_high_speed_bin = []
                                # getting the post-bins
                                post_dp_in_rpm_low_speed_bin = []
                                post_dp_in_rpm_high_speed_bin = []


                                # parsing through all the enteries of the dp-window
                                for j in range(len(pre_dp_window)):  
                                        # if the rpm values are between the bin & (bin+bin_size)

                                        # checking the above condition for pre-rpm window
                                        if pre_rpm_window[j]>= bin and pre_rpm_window[j] < (bin+bin_size):
                                                # getting the dp in the low-speed zone of specific rpm-zone
                                                if int(pre_speed_window[j]) <= SPEED_THRESHOLD:
                                                        pre_dp_in_rpm_low_speed_bin.append(pre_dp_window[j])
                                                # getting the dp in the high-speed zone of specific rpm-zone    
                                                else:
                                                        pre_dp_in_rpm_high_speed_bin.append(pre_dp_window[j])   
                                        
                                        # checking the above condition for the post-window
                                        if post_rpm_window[j]>= bin and post_rpm_window[j] < (bin + bin_size):  
                                        # getting the dp in the low-speed zone of specific rpm-zone
                                                if int(post_speed_window[j]) <= SPEED_THRESHOLD:
                                                        post_dp_in_rpm_low_speed_bin.append(post_dp_window[j])
                                                # getting the dp in the high-speed zone of specific rpm-zone    
                                                else:
                                                        post_dp_in_rpm_high_speed_bin.append(post_dp_window[j])             

                                # if rpm with lower-speed bin
                                if len(pre_dp_in_rpm_low_speed_bin) > 0:
                                        summary_statistics =  pd.DataFrame(pre_dp_in_rpm_low_speed_bin).describe()  
                                        pre_descriptive_statistics_all_bins.append([
                                                                        summary_statistics.loc['count'][0],
                                                                        summary_statistics.loc['mean'][0], 
                                                                        ]) 
                                        pre_rpm_bin_considered.append((bin, (bin+bin_size)))
                                        pre_speed_bin_considered.append("<="+str(SPEED_THRESHOLD))
                                                        
                                if len(post_dp_in_rpm_low_speed_bin) > 0:
                                        summary_statistics =  pd.DataFrame(post_dp_in_rpm_low_speed_bin).describe()  
                                        post_descriptive_statistics_all_bins.append([
                                                                        summary_statistics.loc['count'][0],
                                                                        summary_statistics.loc['mean'][0], 
                                                                       ]) 
                                        post_rpm_bin_considered.append((bin, (bin+bin_size)))
                                        post_speed_bin_considered.append("<="+str(SPEED_THRESHOLD))


                                # if rpm with higher-speed bin                
                                if len(pre_dp_in_rpm_high_speed_bin) > 0:
                                        summary_statistics =  pd.DataFrame(pre_dp_in_rpm_high_speed_bin).describe()  
                                        pre_descriptive_statistics_all_bins.append([
                                                                        summary_statistics.loc['count'][0],
                                                                        summary_statistics.loc['mean'][0]]) 
                                        pre_rpm_bin_considered.append((bin, (bin+bin_size)))
                                        pre_speed_bin_considered.append(">"+str(SPEED_THRESHOLD))

                                if len(post_dp_in_rpm_high_speed_bin) > 0:
                                        summary_statistics =  pd.DataFrame(post_dp_in_rpm_high_speed_bin).describe()  
                                        post_descriptive_statistics_all_bins.append([
                                                                                summary_statistics.loc['count'][0],
                                                                                summary_statistics.loc['mean'][0]]) 
                                        post_rpm_bin_considered.append((bin, (bin+bin_size)))
                                        post_speed_bin_considered.append(">"+str(SPEED_THRESHOLD))   
                                                                          
                                                
                        if len(pre_descriptive_statistics_all_bins):
                                        for j in range(len(pre_descriptive_statistics_all_bins)):  
                                                pre_row_data = []
                                                pre_row_data.append(pre_rpm_bin_considered[j]) #'RPM Bin'
                                                pre_row_data.append(pre_speed_bin_considered[j]) #'Speed Bin'
                                                pre_row_data.extend(pre_descriptive_statistics_all_bins[j]) 
                                                df_pre_statistics_all_vehicles.loc[len(df_pre_statistics_all_vehicles)] = pre_row_data
                                
                        if len(post_descriptive_statistics_all_bins):
                                        for j in range(len(post_descriptive_statistics_all_bins)):
                                                post_row_data = []  
                                                post_row_data.append(post_rpm_bin_considered[j]) #'RPM Bin'
                                                post_row_data.append(post_speed_bin_considered[j]) #'Speed Bin'
                                                post_row_data.extend(post_descriptive_statistics_all_bins[j]) 
                                                df_post_statistics_all_vehicles.loc[len(df_post_statistics_all_vehicles)] = post_row_data

                        # dropping off duplicates from pre and post dataframe
                        df_pre_statistics_all_vehicles = df_pre_statistics_all_vehicles.drop_duplicates()
                        df_post_statistics_all_vehicles = df_post_statistics_all_vehicles.drop_duplicates()

                        # renaming the columns for pre and post statistical dataframes
                        df_pre_statistics_all_vehicles.columns = ["Pre RPM Bin", "Pre Speed Bin", 
                                                                "Pre Count", "Pre Mean"]
                                
                        df_post_statistics_all_vehicles.columns = ["Post RPM Bin", "Post Speed Bin", 
                                                                "Post Count", "Post Mean"]
                                
                        # combining pre and post dataframes basis RPM and SPEED bin values 
                        df_statistics = pd.merge(df_pre_statistics_all_vehicles, df_post_statistics_all_vehicles, how='inner', 
                                                left_on=["Pre RPM Bin", "Pre Speed Bin"],
                                                right_on = ["Post RPM Bin", "Post Speed Bin"])
                                
                        # In case any duplicates generated while combining the two dataframes drop them
                        df_statistics = df_statistics.drop_duplicates()

                        # dropping all those statistical bins wherein the Pre or Post Count of values are less than 5
                        indices_remove = df_statistics[(df_statistics['Pre Count']<5) | (df_statistics['Post Count']<5)].index
                        df_statistics.drop(indices_remove, inplace=True)
                                
                        

                        print("Below given is the statistical DataFrame: ")

                        print(df_statistics)

                        fraction_good_bins = df_statistics[(df_statistics["Post Mean"]<df_statistics["Pre Mean"])].shape[0]/df_statistics.shape[0]

                                
                        print("fraction of good bins: ", fraction_good_bins)
                        # for "high" soot burn
                        # the logic goes something like this
                        # if 0/4 bins has the post-dp < pre-dp, classify as low
                        # if 1/4 or 2/4 has the post-dp < pre-dp, classify as medium 
                        if burn_quality == 'high' and fraction_good_bins == 0:
                                burn_quality = 'low'
                                burn_quality_percentage = burn_quality_percentage/3
                        elif burn_quality == 'high' and fraction_good_bins <= 0.5 and fraction_good_bins != 0:
                                burn_quality = 'medium'
                                burn_quality_percentage = burn_quality_percentage/2   

                        # for "low" soot burn
                        # the logic goes something like this
                        # if 3/4 or 4/4 has the post-dp < pre-dp, classify as medium 
                        if burn_quality == 'low' and fraction_good_bins > 0.5:
                                burn_quality = 'medium'
                                burn_quality_percentage = burn_quality_percentage * 2
        high_speed_count = 0
        # giving the duration and speed status for ecah of the moderate and low burn_quality
        if burn_quality != 'high':
                # if 10 < active_regeneration_duration < Median Duration  --- > Active-Regeneration happened for sufficient duration
                if  active_regeneration_duration>10 and active_regeneration_duration<Median_Duration:
                        duration_status = 'moderate'
                elif  active_regeneration_duration>Median_Duration:
                        duration_status = 'sufficient'       
                # if for atleast for 50% of time; the vehicle is running at required speed for sufficient amount of time         
                for speed in speed_inst_Value:
                        if speed >= SPEED_THRESHOLD:
                                high_speed_count += 1
                if high_speed_count/len(speed_inst_Value) > 0.5:    
                        speed_status = 'sufficient'
                        
                elif  high_speed_count/len(speed_inst_Value) <= 0.5: 
                        speed_status = 'insufficient'
        # if burn_quality is high ; keep the speed and duration status as NA                
        elif burn_quality == 'high':
                speed_status = 'NA'  
                duration_status = 'NA' 

                           

        # return the speed, duration status and the time for which the function is executed
        return speed_status, duration_status, burn_quality_percentage              
                            

        

Playing Ground Starts

In [ ]:
start_time = time.time()

for i in range(2000):
    print(i)

end_time = time.time()

print("the start time is: ", start_time)
print("the end time is: ", end_time)
print("the time required to run the function is: ", (end_time-start_time))

In [ ]:
# 5th May, 08:27 AM  to 5th May, 08:58 AM  'High'
# 3rd May, 08:12 AM to 3rd May, 08:25 AM 'High'
# 3rd May, 07:50 AM to 3rd May, 08:07 AM 'High'
# 3rd May, 07:47 AM to 3rd May, 07:49 AM 'Failed'
# 2nd May, 06:12 PM to 2nd May, 06:30 PM 'High'
# 28th Apr, 07:07 AM to 28th Apr, 07:30 AM  'High'

#AR_start_time = [1714877820000, 1714704120000, 1714702800000, 1714702620000, 1714653720000, 1714268220000]  
#AR_end_time = [1714879680000,1714704900000, 1714703820000, 1714703400000, 1714654800000, 1714269600000]
#BURN_QUALITY = ['High', 'High', 'High', 'Failed', 'Low', 'High']
#BURN_QUALITY_PERCENTAGE = [0.98, 0.7, 0.85, 2, 0.2, 0.67]

AR_start_time = 1715970600000
AR_end_time = 1716057000000
BURN_QUALITY_PERCENTAGE = 0.2

In [ ]:
vehicle_id = 1281011284925480960
FLAG = 'US'
active_regeneration_start_time = AR_start_time
active_regeneration_end_time = AR_end_time
burn_quality_percentage = 0.78
Median_Duration = 20

In [ ]:
Start_TS = active_regeneration_start_time
End_TS = active_regeneration_end_time

URL = 'https://old-data-downloader.intangles-aws-us-east-1.intangles.us/download/' + str(vehicle_id) +  "/" + str(Start_TS) + "/" + str(End_TS)
                

In [ ]:
r = requests.get(URL,stream=True)
OBD_data = r.json() 

In [ ]:
speed_status, duration_status, burn_quality_percentage = regeneration_evidence_v3(vehicle_id, FLAG,
                                        active_regeneration_start_time, active_regeneration_end_time, 
                                        burn_quality_percentage, Median_Duration)

In [ ]:
print(speed_status)
print(duration_status)
print(burn_quality_percentage)

In [ ]:
time_interval_ls = []

ts = AR_start_time
for i in range(100):
    vehicle_id = 1231762781519216640
    FLAG = 'US'
    active_regeneration_start_time = AR_start_time + i*1000
    active_regeneration_end_time = AR_end_time + i*1000
    burn_quality_percentage = BURN_QUALITY_PERCENTAGE
    Median_Duration = 15
    speed_status, duration_status, time_interval= regeneration_evidence_v3(vehicle_id, FLAG,
                                        active_regeneration_start_time, active_regeneration_end_time, 
                                        burn_quality_percentage, Median_Duration)
    time_interval_ls.append(time_interval)
    print(i, " iteration complete")
    print("tim taken in ", i , " iteration is: ", time_interval)

In [ ]:
time_avg = 0
for time in time_interval_ls:
    time_avg += time

In [ ]:
time_avg/len(time_interval_ls)

In [ ]:
pd.DataFrame(time_interval_ls).describe()

In [ ]:
import plotly.express as px

# Create a boxplot using Plotly Express
fig = px.box(x=time_interval_ls)

# Show the plot
fig.show()

In [ ]:
print(speed_status)
print(duration_status)

In [ ]:
print(len(pre_dp_window))
print(len(post_dp_window))

In [ ]:
import matplotlib.pyplot as plt

plt.hist(pre_dp_window)

In [ ]:
import matplotlib.pyplot as plt

plt.hist(post_dp_window)

In [ ]:
speed_status

In [ ]:
duration_status

In [ ]:
regeneration_evidence_v3(vehicle_id, FLAG,
                                        active_regeneration_start_time, active_regeneration_end_time, 
                                        burn_quality_percentage, Median_Duration)

Playing Ground Ends

In [ ]:
speed_status, duration_status= regeneration_evidence_v3(vehicle_id, FLAG,
                                                        active_regeneration_start_time, active_regeneration_end_time, 
                                                        burn_quality_percentage, Median_Duration)

In [ ]:
def regeneration_evidence_v2(pre_dp_window, post_dp_window, 
                          pre_rpm_window, post_rpm_window,
                          pre_speed_window, post_speed_window,
                          SPEED_THRESHOLD, RPM_RANGE, burn_quality,
                          burn_quality_percentage,
                          start_timestamp, end_timestamp):
        
        
        
        # getting the active-regeneration duration
        active_regeneration_duration = (end_timestamp - start_timestamp)//(60*1000)
       
        # if the duration of active-regeneration is less than 5 minutes 
        if active_regeneration_duration < 5:
                evidence = "Insufficient duration"
        # if number of samples for quantification are insuffiecient        
        elif burn_quality == 'NA':
                evidence = "Insufficient evidence"        
        # if burn-quality is either low or high        
        else:
                # if burn quality is either low or high
                if burn_quality == 'low' or burn_quality == 'high':
                        # getting the relevant bin-size & window-size  
                        bin_size = (RPM_RANGE[-1]-RPM_RANGE[0])//2       
                        # getting the columns
                        columns = [
                                'RPM Bin', 'Speed Bin', 'Count', 
                                'Mean', 'Standard Deviation', 'Minimum',
                                '25th Percentile', '50th Percentile',
                                '75th Percentile', 'Maximum'
                                ]

                        # capturing the decriptive statistics, and their corresponsding rpm and speed bin
                        df_pre_statistics_all_vehicles = pd.DataFrame(columns = columns)
                        df_post_statistics_all_vehicles = pd.DataFrame(columns = columns)

                        pre_descriptive_statistics_all_bins = []
                        post_descriptive_statistics_all_bins = []
                        pre_rpm_bin_considered = []
                        post_rpm_bin_considered = []
                        pre_speed_bin_considered = []
                        post_speed_bin_considered = []


                        # getting the RPM_RANGE and creating bin-on those
                        for bin in range(RPM_RANGE[0],  RPM_RANGE[-1], bin_size):
                                # creating the low and high speed bins for each of the rpm-bins
                                # getting the pre-bins
                                pre_dp_in_rpm_low_speed_bin = []
                                pre_dp_in_rpm_high_speed_bin = []
                                # getting the post-bins
                                post_dp_in_rpm_low_speed_bin = []
                                post_dp_in_rpm_high_speed_bin = []


                                # parsing through all the enteries of the dp-window
                                for j in range(len(pre_dp_window)):  
                                        # if the rpm values are between the bin & (bin+bin_size)

                                        # checking the above condition for pre-rpm window
                                        if pre_rpm_window[j]>= bin and pre_rpm_window[j] < (bin+bin_size):
                                                # getting the dp in the low-speed zone of specific rpm-zone
                                                if int(pre_speed_window[j]) <= SPEED_THRESHOLD:
                                                        pre_dp_in_rpm_low_speed_bin.append(pre_dp_window[j])
                                                # getting the dp in the high-speed zone of specific rpm-zone    
                                                else:
                                                    pre_dp_in_rpm_high_speed_bin.append(pre_dp_window[j])   
                                        
                                        # checking the above condition for the post-window
                                        if post_rpm_window[j]>= bin and post_rpm_window[j] < (bin + bin_size):  
                                        # getting the dp in the low-speed zone of specific rpm-zone
                                                if int(post_speed_window[j]) <= SPEED_THRESHOLD:
                                                        post_dp_in_rpm_low_speed_bin.append(post_dp_window[j])
                                                # getting the dp in the high-speed zone of specific rpm-zone    
                                                else:
                                                        post_dp_in_rpm_high_speed_bin.append(post_dp_window[j])             

                                # if rpm with lower-speed bin
                                if len(pre_dp_in_rpm_low_speed_bin) > 0:
                                        summary_statistics =  pd.DataFrame(pre_dp_in_rpm_low_speed_bin).describe()  
                                        pre_descriptive_statistics_all_bins.append([
                                                                        summary_statistics.loc['count'][0],
                                                                        summary_statistics.loc['mean'][0], 
                                                                        summary_statistics.loc['std'][0], 
                                                                        summary_statistics.loc['min'][0], 
                                                                        summary_statistics.loc['25%'][0],
                                                                        summary_statistics.loc['50%'][0], 
                                                                        summary_statistics.loc['75%'][0],
                                                                        summary_statistics.loc['max'][0]]) 
                                        pre_rpm_bin_considered.append((bin, (bin+bin_size)))
                                        pre_speed_bin_considered.append("<="+str(SPEED_THRESHOLD))
                                        
                                if len(post_dp_in_rpm_low_speed_bin) > 0:
                                        summary_statistics =  pd.DataFrame(post_dp_in_rpm_low_speed_bin).describe()  
                                        post_descriptive_statistics_all_bins.append([
                                                                        summary_statistics.loc['count'][0],
                                                                        summary_statistics.loc['mean'][0], 
                                                                        summary_statistics.loc['std'][0], 
                                                                        summary_statistics.loc['min'][0], 
                                                                        summary_statistics.loc['25%'][0],
                                                                        summary_statistics.loc['50%'][0], 
                                                                        summary_statistics.loc['75%'][0],
                                                                        summary_statistics.loc['max'][0]]) 
                                        post_rpm_bin_considered.append((bin, (bin+bin_size)))
                                        post_speed_bin_considered.append("<="+str(SPEED_THRESHOLD))


                                # if rpm with higher-speed bin                
                                if len(pre_dp_in_rpm_high_speed_bin) > 0:
                                        summary_statistics =  pd.DataFrame(pre_dp_in_rpm_high_speed_bin).describe()  
                                        pre_descriptive_statistics_all_bins.append([
                                                                        summary_statistics.loc['count'][0],
                                                                        summary_statistics.loc['mean'][0],
                                                                        summary_statistics.loc['std'][0], 
                                                                        summary_statistics.loc['min'][0], 
                                                                        summary_statistics.loc['25%'][0],
                                                                        summary_statistics.loc['50%'][0], 
                                                                        summary_statistics.loc['75%'][0],
                                                                        summary_statistics.loc['max'][0]]) 
                                        pre_rpm_bin_considered.append((bin, (bin+bin_size)))
                                        pre_speed_bin_considered.append(">"+str(SPEED_THRESHOLD))

                                if len(post_dp_in_rpm_high_speed_bin) > 0:
                                        summary_statistics =  pd.DataFrame(post_dp_in_rpm_high_speed_bin).describe()  
                                        post_descriptive_statistics_all_bins.append([
                                                                                summary_statistics.loc['count'][0],
                                                                                summary_statistics.loc['mean'][0],
                                                                                summary_statistics.loc['std'][0], 
                                                                                summary_statistics.loc['min'][0], 
                                                                                summary_statistics.loc['25%'][0],
                                                                                summary_statistics.loc['50%'][0], 
                                                                                summary_statistics.loc['75%'][0],
                                                                                summary_statistics.loc['max'][0]]) 
                                        post_rpm_bin_considered.append((bin, (bin+bin_size)))
                                        post_speed_bin_considered.append(">"+str(SPEED_THRESHOLD))        
                                                
                                        
                        if len(pre_descriptive_statistics_all_bins):
                                        for j in range(len(pre_descriptive_statistics_all_bins)):  
                                                pre_row_data = []
                                                pre_row_data.append(pre_rpm_bin_considered[j]) #'RPM Bin'
                                                pre_row_data.append(pre_speed_bin_considered[j]) #'Speed Bin'
                                                pre_row_data.extend(pre_descriptive_statistics_all_bins[j]) 
                                        
                                                df_pre_statistics_all_vehicles.loc[len(df_pre_statistics_all_vehicles)] = pre_row_data
                                
                        if len(post_descriptive_statistics_all_bins):
                                        for j in range(len(post_descriptive_statistics_all_bins)):
                                                post_row_data = []  
                                                post_row_data.append(post_rpm_bin_considered[j]) #'RPM Bin'
                                                post_row_data.append(post_speed_bin_considered[j]) #'Speed Bin'
                                                post_row_data.extend(post_descriptive_statistics_all_bins[j]) 
                                                df_post_statistics_all_vehicles.loc[len(df_post_statistics_all_vehicles)] = post_row_data

                        df_pre_statistics_all_vehicles = df_pre_statistics_all_vehicles.drop_duplicates()
                        df_post_statistics_all_vehicles = df_post_statistics_all_vehicles.drop_duplicates()


                        print(df_pre_statistics_all_vehicles)



                        df_pre_statistics_all_vehicles.columns = ["Pre RPM Bin", "Pre Speed Bin", 
                                                                "Pre Count", "Pre Mean", 
                                                                "Pre Standard Deviation", "Pre Minimum",
                                                                "Pre 25th Percentile", "Pre 50th Percentile",
                                                                "Pre 75th Percentile", "Pre Maximum"]
                        
                        df_post_statistics_all_vehicles.columns = ["Post RPM Bin", "Post Speed Bin", 
                                                                "Post Count", "Post Mean", 
                                                                "Post Standard Deviation", "Post Minimum",
                                                                "Post 25th Percentile", "Post 50th Percentile",
                                                                "Post 75th Percentile", "Post Maximum"]
                        
                        df_statistics = pd.merge(df_pre_statistics_all_vehicles, df_post_statistics_all_vehicles, how='inner', 
                                                left_on=["Pre RPM Bin", "Pre Speed Bin"],
                                                right_on = ["Post RPM Bin", "Post Speed Bin"])
                        
                        df_statistics = df_statistics.drop_duplicates()

                        indices_remove = df_statistics[(df_statistics['Pre Count']<5)].index
                        df_statistics.drop(indices_remove, inplace=True)
                        
                        
                        #df_statistics.loc[(df_statistics['Pre Count']<5), ['Pre Mean', 'Post Mean']] = 0 

                        df_statistics["Status"] = "Good"
                        df_statistics.loc[(df_statistics["Post Mean"]>df_statistics["Pre Mean"]), "Status"] = "Bad"
                        
                        c = df_statistics[(df_statistics["Post Mean"]<df_statistics["Pre Mean"])].shape[0]

                        fraction_good_bins = df_statistics[(df_statistics["Post Mean"]<df_statistics["Pre Mean"])].shape[0]/df_statistics.shape[0]

                        
                        print("fraction of good bins: ", fraction_good_bins)
                        # for "high" soot burn
                        # the logic goes something like this
                        # if 0/4 bins has the post-dp < pre-dp, classify as low
                        # if 1/4 or 2/4 has the post-dp < pre-dp, classify as medium 
                        if burn_quality == 'high' and fraction_good_bins == 0:
                                burn_quality = 'low'
                                burn_quality_percentage = burn_quality_percentage/3
                        elif burn_quality == 'high' and fraction_good_bins <= 0.5 and fraction_good_bins != 0:
                                burn_quality = 'medium'
                                burn_quality_percentage = burn_quality_percentage/2   

                         # for "low" soot burn
                        # the logic goes something like this
                        # if 3/4 or 4/4 has the post-dp < pre-dp, classify as medium 
                        if burn_quality == 'low' and fraction_good_bins > 0.5:
                                burn_quality = 'medium'
                                burn_quality_percentage = burn_quality_percentage * 2
                                  
                        if burn_quality == 'low':
                                print(df_statistics)
                                if df_statistics.loc[(df_statistics["Pre Speed Bin"] == str(">")+str(SPEED_THRESHOLD)),"Pre Count"].sum() == 0:
                                                evidence = "Speed conditions were insufficiently met"
                                else:
                                        status_ls = df_statistics.loc[(df_statistics["Pre Speed Bin"] == str(">")+str(SPEED_THRESHOLD)),"Status"].tolist()
                                        any_bad = any(element == "bad" for element in status_ls)
                                        if any_bad:
                                                evidence = "Speed conditions were insufficiently met"
                                        else:
                                                evidence =  "Higher soot load despite speed conditions were met"
                        if burn_quality == 'high':  
                                evidence = "Duration and speed conditions were met"
                elif burn_quality == 'medium' :
                        evidence = "Duration and Speed conditions were moderately met"

        return evidence, burn_quality                        


                
        
        
        


        

In [ ]:
def resample_PID_data(X_Value,Number_of_features):

    # DPFDP has minimum length DATA[1,:], so resample all other variables  w.r.t. DPFDP
    Samples = len(X_Value[1])
    #print(len(X_Value[0]))
    DATA = np.zeros((Number_of_features,Samples))
    TEMP = np.transpose(np.array(X_Value[1]))
 
    for cnt in range(0,Number_of_features):
        if (len(X_Value[cnt]) > 0):
            DATA[cnt,:] = np.transpose(np.array(signal.resample(X_Value[cnt], Samples)))

    DATA[1,:] = TEMP

    return DATA

In [ ]:
def extract_PID_data_v3(data, PROTOCOL,LABEL):
    
    if (PROTOCOL == 'SAE'):

        if LABEL == 'IMAP':
            PID_TAG = '106'
        elif LABEL == 'ENGINE LOAD':
            PID_TAG = '92'
        elif LABEL == 'ENGINE RPM':
            PID_TAG = '190'
        elif LABEL == 'FUEL RATE':
            PID_TAG = '183'
        elif LABEL == 'MAF':
            PID_TAG = '132'
        elif LABEL == 'BOOST':
            PID_TAG = '102'
        elif LABEL == 'BAROMETER':
            PID_TAG = '108'
        elif LABEL == 'THROTTLE':
            PID_TAG = '91'
        elif LABEL == 'ATGMF': # Eaxuast Gas Flow Rate
            PID_TAG = '3236'
        elif LABEL == 'DPFDP': # DPF Diffrential Pressure (across DPF)
            PID_TAG = '3251'
        elif LABEL == 'SCRT':   # SCR Catalyst Temperature Before Catalyst (DPF out)
            PID_TAG = '4360'
        elif LABEL == 'SPEED': # Wheep based Vehicle Speed
            PID_TAG = '84'
        elif LABEL == 'DPFINT':# DPF in Temperature Before DPF (DOC out)
            PID_TAG = '4766' #4766
        elif LABEL == 'IS': #  regen inhibited
            PID_TAG = '3703'        
        elif LABEL == 'FUEL USE': #  Total fuel used (high precision)
            PID_TAG = '5054'        
        elif LABEL == 'DISTANCE': #  Total distance travelled (high precision)
            PID_TAG = '917' # 245
        elif LABEL == 'SOOTLOAD':
            PID_TAG = '5466' # 245 #3719 generic check       
        elif LABEL == 'ACTIVEREGEN':
            PID_TAG = '3700' # 245

    elif (PROTOCOL == 'SAE_AVG'):

        if LABEL == 'IMAP':
            PID_TAG = 'spn_106_avg'
        elif LABEL == 'ENGINE LOAD':
            PID_TAG = 'spn_92_avg'
        elif LABEL == 'ENGINE RPM':
            PID_TAG = 'spn_190_avg'
        elif LABEL == 'FUEL RATE':
            PID_TAG = 'spn_183_avg'
        elif LABEL == 'MAF':
            PID_TAG = 'spn_132_avg'
        elif LABEL == 'BOOST':
            PID_TAG = 'spn_102_avg'
        elif LABEL == 'BAROMETER':
            PID_TAG = 'spn_108_avg'
        elif LABEL == 'THROTTLE':
            PID_TAG = 'spn_91_avg'
        elif LABEL == 'ATGMF': # Eaxuast Gas Flow Rate
            PID_TAG = 'spn_3236_avg'
        elif LABEL == 'DPFDP': # DPF Diffrential Pressure (across DPF)
            PID_TAG = 'spn_3251_avg'
        elif LABEL == 'SCRT':   # SCR Catalyst Temperature Before Catalyst (DPF out)
            PID_TAG = 'spn_4360_avg'
        elif LABEL == 'SPEED': # Wheep based Vehicle Speed
            PID_TAG = 'spn_84_avg'
        elif LABEL == 'DPFINT':# DPF in Temperature Before DPF (DOC out)
            PID_TAG = 'spn_4766_avg' #4766
        elif LABEL == 'IS': #  regen inhibited
            PID_TAG = 'spn_3703_avg'        
        elif LABEL == 'FUEL USE': #  Total fuel used (high precision)
            PID_TAG = 'spn_5054_avg'        
        elif LABEL == 'DISTANCE': #  Total distance travelled (high precision)
            PID_TAG = 'spn_917_avg' # 245
        elif LABEL == 'SOOTLOAD':
            PID_TAG = 'spn_5466_avg' # 245 #3719 generic check       
        elif LABEL == 'ACTIVEREGEN':
            PID_TAG = 'spn_3700_avg' # 245
        

    elif(PROTOCOL == 'ISO'):

        if LABEL == 'IMAP':
            PID_TAG = '87BC'#MODIFY the "if PID_TAG in State1:" loop (append for loop on top)  
        elif LABEL == 'ENGINE LOAD':
            PID_TAG = '04'
        elif LABEL == 'ENGINE RPM':
            PID_TAG = '0C'
        elif LABEL == 'FUEL RATE':
            PID_TAG = '5E'
        elif LABEL == 'MAF':
            PID_TAG = '10'
        elif LABEL == 'BOOST':
            PID_TAG = '102'
        elif LABEL == 'BAROMETER':
            PID_TAG = '33'
        elif LABEL == 'THROTTLE':
            PID_TAG = '49'#MODIFY the "if PID_TAG in State1:" loop (append for loop on top)
        elif LABEL == 'DPFDP': # DPF Diffrential Pressure (across DPF)
            PID_TAG = '7A'
    Time_vec = []
    Val_vec = []


    for i in range(len(data)):
        if "pids" in data[i]:
            if len(data[i]['pids'])>0:
                for sub_pid_cnt in range(0,len(data[i]['pids'])):
                    State = data[i]['pids'][sub_pid_cnt]
                    if PID_TAG in State:
                        Time_vec.append(State[PID_TAG]['timestamp'])
                        Val_vec.append(State[PID_TAG]['value'][0])
      
                
    return Time_vec,Val_vec        
            

In [ ]:
PATH = "D:/products/time_series/DPF_2.0_TESTING_DATA/1231762781519216640_DP_included/"
VARIABLES = ['ENGINE RPM', 'DPFDP', 'SPEED']

# dir_list = os.listdir(PATH)
# print(dir_list)
# DATA = np.array([], dtype=np.int64).reshape(len(VARIABLES),0)
#for data_packet_cnt in range(0,len(dir_list)):
#OBD_data_path = PATH + dir_list[data_packet_cnt]
path = "D:/products/time_series/DPF_2.0_TESTING_DATA/1231762781519216640/" 
#OBD_data_path = "D:/products/time_series/DPF_2.0_TESTING_DATA/1231762781519216640_1709231400000_1714933800000/5TH_MY_REGEN/1231762781519216640_1714897426000_1714933800000.json"
#OBD_data = [json.loads(line) for line in open(OBD_data_path, 'r')]

In [ ]:
vehicle = 1231762781519216640
AR_start_time = [1714877820000, 1714704120000, 1714702800000, 1714702620000, 1714653720000, 1714268220000]  
AR_end_time = [1714879680000,1714704900000, 1714703820000, 1714702740000, 1714654800000, 1714269600000]
for i in range(len(AR_start_time)):
    print(AR_start_time[i])
    print(AR_end_time[i])
    Start_TS = AR_start_time[i] - 1*60*60*1000
    End_TS = AR_end_time[i] + 1*60*60*1000
    print("this is the starting timestamp ", Start_TS)
    print("this is the ending timestamp ", End_TS)
    URL = ' https://old-data-downloader.intangles-aws-us-east-1.intangles.us/download/' + str(vehicle) +  "/" + str(Start_TS) + "/" + str(End_TS)
    r = requests.get(URL,stream=True)
    OBD_data = r.json() 
    print(OBD_data[0])
    dp_inst_Time, dp_inst_Value = extract_PID_data_v3(OBD_data, 'SAE_AVG', 'DPFDP')
    rpm_inst_Time, rpm_inst_Value = extract_PID_data_v3(OBD_data, 'SAE_AVG', 'ENGINE RPM')
    throttle_inst_Time, throttle_inst_Value = extract_PID_data_v3(OBD_data, 'SAE_AVG', 'THROTTLE')
    break

In [ ]:
dp_inst_Time[-1]

In [ ]:
dp_rpm_constrained = []
throttle_rpm_constrained = []
time_rpm_constrained = []
rpm_constrained = []
for i in range(len(dp_inst_Time)):
    if rpm_inst_Value[i]>=1000 and rpm_inst_Value[i]<=2000:
        dp_rpm_constrained.append(dp_inst_Value[i])
        time_rpm_constrained.append(dp_inst_Time[i])
        rpm_constrained.append(rpm_inst_Value[i])
        throttle_rpm_constrained.append(throttle_inst_Value[i])


In [ ]:
#plt.hist(throttle_rpm_constrained)
higher_throttle = []
for i in range(len(throttle_rpm_constrained)):
    if throttle_rpm_constrained[i]>=30:
        higher_throttle.append(throttle_rpm_constrained[i])
 


In [ ]:
len(higher_throttle)/len(throttle_rpm_constrained)

In [ ]:
dp_rpm_throttle_constrained = []
time_rpm_throttle_constrained = []
rpm_throttle_constrained = []
for i in range(len(dp_rpm_constrained)):
    if throttle_inst_Value[i]>=50:
        dp_rpm_throttle_constrained.append(dp_inst_Value[i])
        time_rpm_throttle_constrained.append(dp_inst_Time[i])
        rpm_throttle_constrained.append(rpm_inst_Value[i])


In [ ]:
_, start_idx = find_nearest(time_rpm_throttle_constrained, AR_start_time[0])
_, end_idx = find_nearest(time_rpm_throttle_constrained, AR_end_time[0])
_, last_idx = find_nearest(time_rpm_throttle_constrained, AR_end_time[0] + 60*60*1000)
mid_idx = (end_idx + start_idx)//2
print(mid_idx - start_idx + 1)
print(last_idx-end_idx+1)

In [ ]:
plt.hist(rpm_throttle_constrained)

In [ ]:
plt.hist(rpm_constrained)

In [ ]:
pre_rpm = rpm_throttle_constrained[:mid_idx]
post_rpm = rpm_throttle_constrained[end_idx:]

In [ ]:
# dp_rpm_throttle_constrained.append(dp_inst_Value[i])
# time_rpm_throttle_constrained.append(dp_inst_Time[i])
# rpm_throttle_constrained.append(rpm_inst_Value[i])


In [ ]:
pre_low_rpm = []
post_low_rpm = []
pre_high_rpm = []
post_high_rpm = []
for i in range(len(pre_rpm)):
    if pre_rpm[i]>=1000 and pre_rpm[i]<1400:
        pre_low_rpm.append(pre_rpm[i])
    elif pre_rpm[i]>=1400 and pre_rpm[i]<=2000:
        pre_high_rpm.append(pre_rpm[i])
for i in range(len(post_rpm)):
    if post_rpm[i]>=1000 and post_rpm[i]<1400:
        post_low_rpm.append(post_rpm[i])
    elif post_rpm[i]>=1400 and post_rpm[i]<=2000:
        post_high_rpm.append(post_rpm[i])        



In [ ]:
print(len(pre_low_rpm))
print(len(pre_high_rpm))
print(len(post_low_rpm))
print(len(post_high_rpm))

Checking of the logic done above

In [ ]:
Time = []
rpm_Val = []
speed_Val = []
dp_Val= []

dir_packets = os.listdir(path)
dir_packets.sort()
for packet in dir_packets:
    dir_path = path + str(packet)
    OBD_data = [json.loads(line) for line in open(dir_path, 'r')]
    if len(OBD_data[0]['results']['data']):
        rpm_inst_Time, rpm_inst_Value = extract_PID_data_v3(OBD_data[0]['results']['data'], 'SAE_AVG', 'ENGINE RPM')
        dp_inst_Time, dp_inst_Value = extract_PID_data_v3(OBD_data[0]['results']['data'], 'SAE_AVG', 'DPFDP')
        speed_inst_Time, speed_inst_Value = extract_PID_data_v3(OBD_data[0]['results']['data'], 'SAE_AVG', 'SPEED')
        Time.extend(rpm_inst_Time)
        rpm_Val.extend(rpm_inst_Value)
        speed_Val.extend(speed_inst_Value)
        dp_Val.extend(dp_inst_Value)
        




In [ ]:
import plotly.graph_objects as go

# Create some data
x = np.arange(len(dp_Val))
y = dp_Val

# Create a Plotly figure
fig = go.Figure(data=go.Scatter(x=x, y=y, mode='markers', marker=dict(color='blue')))

# Update layout
fig.update_layout(title='DP-Value vs Engine-Hours', xaxis_title='Engine-Hours', yaxis_title='DP Value')

# Show the figure
fig.show()

In [ ]:
# applying RPM constraint on the DP-Values
dp_value_rpm_constrained = []
rpm_value_constrained = []
speed_value_rpm_constrained = []
time_constrained = []
for i in range(len(dp_Val)):
    if rpm_Val[i]>=1000 and rpm_Val[i]<=2000:
        rpm_value_constrained.append(rpm_Val[i])
        dp_value_rpm_constrained.append(dp_Val[i])
        speed_value_rpm_constrained.append(speed_Val[i])
        time_constrained.append(Time[i])





In [ ]:
import plotly.graph_objects as go

# Create some data
x = np.arange(len(dp_value_rpm_constrained))
y = dp_value_rpm_constrained

# Create a Plotly figure
fig = go.Figure(data=go.Scatter(x=x, y=y, mode='markers', marker=dict(color='blue')))

# Update layout
fig.update_layout(title='DP-Value Constrained vs Engine-Hours', xaxis_title='Engine-Hours', yaxis_title='DP Value')

# Show the figure
fig.show()

In [ ]:
# Applying RPN constraint to see how the data looks like after that
# 1000 to 2000
col=[]
for i in range(0,len(dp_value_rpm_constrained)):
    if 1000<=rpm_value_constrained[i] and rpm_value_constrained[i]<1500:
        col.append('[(1000, 1500)] or [lowrpm]')   
    elif rpm_value_constrained[i]>=1500 and rpm_value_constrained[i]<2000:
        col.append('[(1500, 2000)] or [highrpm]')
data = {'Engine_Hours': np.arange(len(dp_value_rpm_constrained)), 'DP_Value': dp_value_rpm_constrained}
df = pd.DataFrame(data)
fig =px.scatter(df, x='Engine_Hours', y='DP_Value', color=col, color_discrete_map={'[(1000, 1500)] or [lowrpm]':'blue',
                                                                                    '[(1500, 2000)] or [highrpm]': 'orange',
                                                                                       }) 
fig.show()                                                                                           
   

In [ ]:
# Applying RPN constraint to see how the data looks like after that
# 1000 to 2000 and 80 are RPM and speed thresholds respectively
import plotly.express as px
col=[]
for i in range(0,len(dp_value_rpm_constrained)):
    if (1000<=rpm_value_constrained[i] and rpm_value_constrained[i]<1500) and speed_value_rpm_constrained[i]<=80:
        col.append('[(1000, 1500), <=80] or [lowrpm, lowspeed]')
    elif (1000<=rpm_value_constrained[i] and rpm_value_constrained[i]<1500) and speed_value_rpm_constrained[i]>80:
        col.append('[(1000, 1500), >80] or [lowrpm, lowspeed]')       
    elif (rpm_value_constrained[i]>=1500 and rpm_value_constrained[i]<2000) and speed_value_rpm_constrained[i]<=80:
        col.append('[(1500, 2000), <=80] or [highrpm, lowspeed]')
    elif (rpm_value_constrained[i]>=1500 and rpm_value_constrained[i]<2000) and speed_value_rpm_constrained[i]>80:
        col.append('[(1500, 2000), >80] or [highrpm, lowspeed]')    
data = {'Engine_Hours': np.arange(len(dp_value_rpm_constrained)), 'DP_Value': dp_value_rpm_constrained}
df = pd.DataFrame(data)
fig =px.scatter(df, x='Engine_Hours', y='DP_Value', color=col, color_discrete_map={'[(1000, 1500), <=80] or [lowrpm, lowspeed]':'blue',
                                                                                    '[(1000, 1500), >80] or [lowrpm, lowspeed]': 'green',
                                                                                    '[(1500, 2000), <=80] or [highrpm, lowspeed]': 'orange',
                                                                                    '[(1500, 2000), >80] or [highrpm, lowspeed]': 'purple'
                                                                                       }) 
fig.show()    

In [ ]:
AR_start_time = [1714877820000, 1714704120000, 1714702800000, 1714702620000, 1714653720000, 1714268220000]  
AR_end_time = [1714879680000,1714704900000, 1714703820000, 1714702740000, 1714654800000, 1714269600000]
BURN_QUALITY = ['high', 'high', 'high', 'low', 'high', 'high']
for i in range(len(AR_start_time)):
       print(BURN_QUALITY[i])
       _, start_idx = find_nearest(time_constrained, AR_start_time[i])
       _, end_idx = find_nearest(time_constrained, AR_end_time[i])
       mid_idx = (start_idx + end_idx)//2
       pre_dp_window = dp_value_rpm_constrained[mid_idx-50:mid_idx]
       post_dp_window = dp_value_rpm_constrained[end_idx: end_idx+50]

       pre_rpm_window = rpm_value_constrained[mid_idx-50:mid_idx]
       post_rpm_window = rpm_value_constrained[end_idx: end_idx+50]

       pre_speed_window = speed_value_rpm_constrained[mid_idx-50:mid_idx]
       post_speed_window = speed_value_rpm_constrained[end_idx: end_idx+50]

       SPEED_THRESHOLD = 80
       RPM_RANGE = [1000, 2000]
       burn_quality = BURN_QUALITY[i]
       burn_quality_percentage = 0.8
       start_timestamp = AR_start_time[i]
       end_timestamp = AR_end_time[i]
       
       evidence, burn_quality = regeneration_evidence_v2(pre_dp_window, post_dp_window, 
                          pre_rpm_window, post_rpm_window,
                          pre_speed_window, post_speed_window,
                          SPEED_THRESHOLD, RPM_RANGE, burn_quality,
                          burn_quality_percentage,
                          start_timestamp, end_timestamp)
       print("this is the evidence: ", evidence)                   
       print(i)                   
       #break                   

In [ ]:
evidence

In [ ]:
df_statistics, evidence = regeneration_evidence_v2(pre_dp_window, post_dp_window, 
                          pre_rpm_window, post_rpm_window,
                          pre_speed_window, post_speed_window,
                          SPEED_THRESHOLD, RPM_RANGE, burn_quality,
                          burn_quality_percentage,
                          start_timestamp, end_timestamp)

In [ ]:
# Applying RPN constraint to see how the data looks like after that
# 1000 to 2000
col=[]
        for i in range(0,len(X_2[1,:])):
            if 1000<=X_2[0][i] and X_2[0][i]<1500 and X_2[5][i]>40:
                col.append('[(1000, 1500) & >40] or [lowrpm, highspeed]')
            elif 1000<=X_2[0][i] and X_2[0][i]<1500 and X_2[5][i]<=40:
                col.append('[(1000, 1500) & <=40] or [lowrpm, lowspeed]')    
            elif X_2[0][i]>=1500 and X_2[0][i]<2000 and X_2[5][i]>40:
                col.append('[(1500, 2000) & >40] or [highrpm, highspeed]')
            elif X_2[0][i]>=1500 and X_2[0][i]<2000 and X_2[5][i]<=40:
                col.append('[(1500, 2000) & <=40] or [highrpm, lowspeed]')   
                

        data = {'Engine_Hours': np.arange(len(T_2)), 'DP_Value': X_2[1,:]}
        df = pd.DataFrame(data)    
        #utc_Time = [datetime.utcfromtimestamp(ms / 1000).strftime('%Y-%m-%d %H:%M:%S') for ms in T_2] 

        # fig = px.scatter(df, x='utc_Time', y='DP_Value', color=col, color_discrete_map={'(1000, 1500)': 'green', '(1500, 2000)': 'yellow'})
        # fig.add_trace(go.Scatter(x=np.arange(len(X_4[1])), y=regeneration_binary_signal, mode='markers', name='Regeneration Signal', marker=dict(color='red')))
        fig = px.scatter(df, x='Engine_Hours', y='DP_Value', color=col, color_discrete_map={'[(1000, 1500) & >40] or [lowrpm, highspeed]':'green',
                                                                                        '[(1000, 1500) & <=40] or [lowrpm, lowspeed]':'blue',
                                                                                        '[(1500, 2000) & >40] or [highrpm, highspeed]': 'orange',
                                                                                        '[(1500, 2000) & <=40] or [highrpm, lowspeed]': 'purple'})
       fig.update_layout(
            title={
                'text': str(df_vector_to_be_explored1.loc[(df_vector_to_be_explored1["Vehicle_ID"]==int(vehicle)) & 
                                                                 (df_vector_to_be_explored1["Active Regeneration Start Time"]==start_regeneration_time[start_idx])
                                                                 &  (df_vector_to_be_explored1["Active Regeneration End Time"]==end_regeneration_time[start_idx]),
                                                                 'burn_quantification' ].values[0]) + " soot" + " burn"
                                                                 ,
                'y': 0.9,  # Adjust the vertical position
                'x': 0.5,  # Adjust the horizontal position
                'xanchor': 'center',
                'yanchor': 'top',
                'font': dict(
                    family="Arial",
                    size=24,
                    color="black"
                )
            }
        )

        fig.show()
        

In [ ]:
# 5th May, 08:27 AM  to 5th May, 08:58 AM  'High'
# 3rd May, 08:12 AM to 3rd May, 08:25 AM 'High'
# 3rd May, 07:50 AM to 3rd May, 08:07 AM 'High'
# 3rd May, 07:47 AM to 3rd May, 07:49 AM 'Failed'
# 2nd May, 06:12 PM to 2nd May, 06:30 PM 'High'
# 28th Apr, 07:07 AM to 28th Apr, 07:30 AM  'High'
AR_start_time = [1714877820000, 1714704120000, 1714702800000, 1714702620000, 1714653720000, 1714268220000]  
AR_end_time = [1714879680000,1714704900000, 1714703820000, 1714702740000, 1714654800000, 1714269600000]
BURN_QUALITY = ['High', 'High', 'High', 'Failed', 'High', 'High']

In [ ]:
for i in range(len(AR_start_time)):
    regeneration_start_idx = Time.index(AR_start_time[i])
    regeneration_end_idx = Time.index(AR_end_time[i])
    mid_idx = (regeneration_start_idx + regeneration_end_idx)//2
    pre_dp_window = dp_Val[(mid_idx - 50):mid_idx]
    pre_rpm_window 

In [ ]:
OBD_data[0]['results']

In [ ]:
VARIABLES = ['ENGINE RPM', 'DPFDP', 'SPEED']
Time_vec,Val_vec = extract_PID_data_v3(OBD_data[0]['results']['data'], 'SAE', 'DPFDP')
rpm_Time = []
rpm_Val = []

dp_Time = []
dp_Val = []

speed_Time = []
speed_Val = []
for var_type in VARIABLES:
    X_Time, X_Value = extract_PID_data_v3(OBD_data[0]['results']['data'],'SAE_AVG',var_type)
    if var_type == 'ENGINE RPM':
        rpm_Time.extend(X_Time)
        rpm_Val.extend(X_Value)
    elif var_type == 'DPFDP':
        dp_Time.extend(X_Time)
        dp_Val.extend(X_Value)  
    elif var_type == 'SPEED':
        speed_Time.extend(X_Time)
        speed_Val.extend(X_Value)        

In [ ]:
print("rpm frequency")
ls_rpm_frequency = []
for i in range(1, len(rpm_Time)):
    ls_rpm_frequency.append((rpm_Time[i]-rpm_Time[i-1])/(1000))


In [ ]:
print("dp frequency")
ls_dp_frequency = []
for i in range(1, len(dp_Time)):
    ls_dp_frequency.append((dp_Time[i]-dp_Time[i-1])/(1000))

In [ ]:
print("speed frequency")
ls_speed_frequency = []
for i in range(1, len(dp_Time)):
    ls_speed_frequency.append((dp_Time[i]-dp_Time[i-1])/(1000))

In [ ]:
active_regeneration_start = 1714877820000
active_regeneration_end = 1714879680000

_, active_regeneration_start_idx =  find_nearest(dp_Time, active_regeneration_start)
_, active_regeneration_end_idx =  find_nearest(dp_Time, active_regeneration_end)


In [ ]:
mid_idx = (active_regeneration_start_idx + active_regeneration_end_idx)//2

In [ ]:
mid_idx

In [ ]:
# pre_dp_window, post_dp_window, 
# pre_rpm_window, post_rpm_window,
# pre_speed_window, post_speed_window,
# SPEED_THRESHOLD, RPM_RANGE, burn_quality,
# burn_quality_percentage,
# start_timestamp, end_timestamp
pre_dp_window = dp_Val[(mid_idx - 50): mid_idx]
pre_rpm_window = rpm_Val[(mid_idx - 50): mid_idx]
pre_speed_window = speed_Val[(mid_idx - 50): mid_idx]


In [ ]:
pre_dp_window

Ignore

In [ ]:
def resample_PID_data(X_Value,Number_of_features):

    # DPFDP has minimum length DATA[1,:], so resample all other variables  w.r.t. DPFDP
    Samples = len(X_Value[1])
    #print(len(X_Value[0]))
    DATA = np.zeros((Number_of_features,Samples))
    TEMP = np.transpose(np.array(X_Value[1]))
 
    for cnt in range(0,Number_of_features):
        if (len(X_Value[cnt]) > 0):
            DATA[cnt,:] = np.transpose(np.array(signal.resample(X_Value[cnt], Samples)))

    DATA[1,:] = TEMP

    return DATA

In [ ]:
PATH = "D:/products/time_series/DPF_2.0_TESTING_DATA/1231762781519216640_1709231400000_1714933800000/5TH_MY_REGEN/"

In [ ]:
dir_list = os.listdir(PATH)
print(dir_list)
DATA = np.array([], dtype=np.int64).reshape(len(VARIABLES),0)
for data_packet_cnt in range(0,len(dir_list)):
#for data_packet_cnt in range(0,5):
    OBD_data_path = PATH + dir_list[data_packet_cnt] 
    OBD_data = [json.loads(line) for line in open(OBD_data_path, 'r')]

In [ ]:
#VARIABLES = ['ENGINE RPM', 'DPFDP']
VARIABLES = ['ENGINE RPM']

In [ ]:
DATA = np.array([], dtype=np.int64).reshape(len(VARIABLES),0)
T_L = []
V_L = []
T1 = []
for var_type in VARIABLES:
    X_Time, X_Value = extract_PID_data_v3(OBD_data[0]['results']['data'],'SAE',var_type)
    T_L.append(np.array((X_Time), dtype=np.int64))
    V_L.append(np.array((X_Value), dtype=float)) 



# if (len(V_L[0]) > 0):
#     Temp_time = np.array(T_L[1]) # its DP time so no need to resample
#     Resample_data = resample_PID_data(V_L,len(VARIABLES)) # cross check by plot
    
#     DATA = np.concatenate((DATA,Resample_data),axis=1)
#     T1 = np.concatenate((T1,Temp_time),axis=0)

In [ ]:
T1.sort()

In [ ]:
X_Time[0]

In [ ]:
X_Time[-1]

In [2]:
from datetime import datetime, timezone

def convert_ms_to_utc(ms):
    # Convert milliseconds to seconds
    seconds = ms / 1000.0
    # Create a datetime object from the seconds, with UTC timezone
    utc_time = datetime.utcfromtimestamp(seconds).replace(tzinfo=timezone.utc)
    return utc_time

milliseconds = 1621416523000  # Example milliseconds

utc_time = convert_ms_to_utc(milliseconds)
print("UTC time:", utc_time)


UTC time: 2021-05-19 09:28:43+00:00


In [3]:
from datetime import datetime, timezone

def convert_utc_to_ms(utc_time):
    # Convert UTC time to POSIX timestamp
    posix_timestamp = utc_time.timestamp()
    # Convert POSIX timestamp to milliseconds
    milliseconds = int(posix_timestamp * 1000)
    return milliseconds

# Example UTC time
utc_time = datetime.utcnow().replace(tzinfo=timezone.utc)
utc_time = datetime.fromisoformat('2024-05-17 16:56:30+00:00')


milliseconds = convert_utc_to_ms(utc_time)
print("Milliseconds since the epoch:", milliseconds)


Milliseconds since the epoch: 1715964990000


Plotting the DPF associated plots

In [4]:
# function to extract the required PIDs from the string LABELS
def extract_PID_data(data, PROTOCOL,LABEL):
    
    if (PROTOCOL == 'SAE'):

        if LABEL == 'ENGINE LOAD': # Engine Load
            PID_TAG = 'spn_92_avg'
        elif LABEL == 'ENGINE RPM': # Engine Speed (RPM)
            PID_TAG = 'spn_190_avg'
        elif LABEL == 'DPFDP': # DPF Diffrential Pressure (across DPF)
            PID_TAG = 'spn_3251_avg'
        elif LABEL == 'SPEED': # Wheep based Vehicle Speed
            PID_TAG = 'spn_84_avg'
        elif LABEL == 'DPFINT':# DPF in Temperature Before DPF (DOC out)
            PID_TAG = 'spn_3250_avg' 
        elif LABEL == 'SOOTLOAD_3719': #  Soot load  
            PID_TAG = 'spn_3719_avg' 
        elif LABEL == 'SOOTLOAD_5466': # Soot load (Regen threshold)    
            PID_TAG = 'spn_5466_avg'      
        elif LABEL == 'ACTIVEREGEN':  # Regen status
            PID_TAG = 'spn_3700_avg' 
                  

    Time_vec = []
    Val_vec = []
    
    for data_cnt in range(0,len(data)):
        if "pids" in data[data_cnt]:
            if len(data[data_cnt]['pids'])>0:
                for sub_pid_cnt in range(0,len(data[data_cnt]['pids'])):  #this loop
                    State = data[data_cnt]['pids'][sub_pid_cnt]
                    if PID_TAG in State:
                        #print("--------------------------------------------IN----------------------------------")
                        Time_vec.append(np.array(State[PID_TAG]['timestamp'], dtype=np.int64))
                        Val_vec.append(np.array(State[PID_TAG]['value'], dtype=float))
      
    return Time_vec,Val_vec

In [1]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import requests
import numpy as np
import datetime 

In [2]:
vehicle_id = '1318345298031935488' #T880 Dump Truck Tier 3 (Cummins X15) 2015
Start_TS =  1723939200000
End_TS = 1724284799000
OBD_data_path = 'https://old-data-downloader.intangles-aws-us-east-1.intangles.us/download/' + str(vehicle_id) +  "/" + str(Start_TS) + "/" + str(End_TS)
r = requests.get(OBD_data_path, stream=True)
OBD_data = r.json()  


In [5]:
# Create the figure
fig = make_subplots(rows=7, cols=1, shared_xaxes=True)
X_Time, X_Value = extract_PID_data(OBD_data,'SAE','DPFDP')
DP_val = []
TS = []
for cnt in range(0,len(X_Time)):
    TEMP = datetime.datetime.fromtimestamp((X_Time[cnt] + 19800*1000) / 1000.0, tz=datetime.timezone.utc)
    TS.append(TEMP)
    DP_val.append(X_Value[cnt][0])
# Add the first trace
fig.add_trace(go.Scatter(x=TS, y=DP_val, name='DPFDP', mode='markers' ), row=1, col=1 )
fig.add_trace(go.Scatter(x=TS, y=[7]*len(TS), name='DPFDP Threshold', mode='lines', line=dict(color='red', dash='dash')), row=1, col=1 )
X_Time, X_Value = extract_PID_data(OBD_data,'SAE','DPFINT')
Temp_val = []
TS = []
for cnt in range(0,len(X_Time)):
    TEMP = datetime.datetime.fromtimestamp((X_Time[cnt] + 19800*1000) / 1000.0, tz=datetime.timezone.utc)
    TS.append(TEMP)
    Temp_val.append(X_Value[cnt][0])
# Add the second trace
fig.add_trace(go.Scatter(x=TS, y=Temp_val, name='DPFINT', mode='markers'), row=2, col=1)
fig.add_trace(go.Scatter(x=TS, y=[500]*len(TS), name='DPFINT Threshold', mode='lines', line=dict(color='red', dash='dash')), row=2, col=1 )
X_Time, X_Value = extract_PID_data(OBD_data,'SAE','ENGINE RPM')
Speed_val = []
TS = []
for cnt in range(0,len(X_Time)):
    TEMP = datetime.datetime.fromtimestamp((X_Time[cnt] + 19800*1000) / 1000.0, tz=datetime.timezone.utc)
    TS.append(TEMP)
    Speed_val.append(X_Value[cnt][0])
# Add the second trace
fig.add_trace(go.Scatter(x=TS, y=Speed_val, name='ENGINE RPM', mode='markers'), row=3, col=1)
X_Time, X_Value = extract_PID_data(OBD_data,'SAE','SPEED')
Speed_val = []
TS = []
for cnt in range(0,len(X_Time)):
    TEMP = datetime.datetime.fromtimestamp((X_Time[cnt] + 19800*1000) / 1000.0, tz=datetime.timezone.utc)
    TS.append(TEMP)
    Speed_val.append(X_Value[cnt][0])
# Add the second trace
fig.add_trace(go.Scatter(x=TS, y=Speed_val, name='SPEED', mode='markers'), row=4, col=1)
X_Time, X_Value = extract_PID_data(OBD_data,'SAE','SOOTLOAD_5466')
AR_val = []
TS = []
for cnt in range(0,len(X_Time)):
    TEMP = datetime.datetime.fromtimestamp((X_Time[cnt] + 19800*1000) / 1000.0, tz=datetime.timezone.utc)
    TS.append(TEMP)
    AR_val.append(X_Value[cnt][0])
# Add the second trace
fig.add_trace(go.Scatter(x=TS, y=AR_val, name='SOOTLOAD_5466', mode='markers'), row=5, col=1)
X_Time, X_Value = extract_PID_data(OBD_data,'SAE','SOOTLOAD_3719')
AR_val = []
TS = []
for cnt in range(0,len(X_Time)):
    TEMP = datetime.datetime.fromtimestamp((X_Time[cnt] + 19800*1000) / 1000.0, tz=datetime.timezone.utc)
    TS.append(TEMP)
    AR_val.append(X_Value[cnt][0])
# Add the second trace
fig.add_trace(go.Scatter(x=TS, y=AR_val, name='SOOTLOAD_3719', mode='markers'), row=6, col=1)
X_Time, X_Value = extract_PID_data(OBD_data,'SAE','ACTIVEREGEN')
AR_val = []
TS = []
for cnt in range(0,len(X_Time)):
    TEMP = datetime.datetime.fromtimestamp((X_Time[cnt] + 19800*1000) / 1000.0, tz=datetime.timezone.utc)
    TS.append(TEMP)
    AR_val.append(X_Value[cnt][0])
# Add the second trace
fig.add_trace(go.Scatter(x=TS, y=AR_val, name='ACTIVEREGEN', mode='markers'), row=7, col=1)
# Set the y-axis titles
fig.update_layout(title=f"Vehicle ID: {vehicle_id}")

#fig.show()

fig.write_html("D:/DPF-2.0/" + str(vehicle_id) + "_" + str(Start_TS) + "_" + str(End_TS) + ".html")
